In [1]:
import numpy as np

In [3]:
fnames_train.shape, fnames_test.shape

((60,), (60,))

In [4]:
import get_iterations
reload(get_iterations)

<module 'get_iterations' from 'get_iterations.pyc'>

In [5]:
iter_train, train_evts = get_iterations.get_n_iterations(fnames_train)
print(iter_train, train_evts)
iter_test, test_evts = get_iterations.get_n_iterations(fnames_test)
print(iter_test, test_evts)

(1986, 125153)
(534, 31318)


In [6]:
from keras.layers import Input
from keras.layers import concatenate, add
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.core import Flatten, Dense

from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

def vgg_separate_channels(num_classes, optimizer, include_dense=False):
    """VGG inspired Convolutional Networks with multiple inputs
    
    Parameters
    ----------
    num_classes : int
        Number of classes to predict
            
    optimizer : keras.optimizers.Optimizer
        Instance of Keras optimizer to attach to the resulting network
        
    include_dense : bool (default False)
        Flag indicating wether to include a Dense Layer at the top 
        of each separate vgg-like sub-network.
    """
    
    def _conv_block(input_layer, include_dense=False):
        """VGG-inspired Convolutional Blocks"""
        
        prefix = input_layer.name.split('_')[0]

        # Block 1
        x = Conv2D(32, kernel_size=(3, 3), activation='relu', 
                   padding='same', name='{}_block1_conv1'.format(prefix))(input_layer)
        
        x = Conv2D(32, kernel_size=(3, 3), activation='relu', 
                   padding='same', name='{}_block1_conv2'.format(prefix))(x)
        
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), 
                         padding='same', name='{}_block1_pool'.format(prefix))(x)

        # Block 2
        x = Conv2D(64, kernel_size=(3, 3), activation='relu', 
                   padding='same', name='{}_block2_conv1'.format(prefix))(x)
        
        x = Conv2D(64, kernel_size=(3, 3), activation='relu', 
                   padding='same', name='{}_block2_conv2'.format(prefix))(x)
        
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), 
                         padding='same', name='{}_block2_pool'.format(prefix))(x)

        # Block 3
        x = Conv2D(128, kernel_size=(3, 3), activation='relu', 
                   padding='same', name='{}_block3_conv2'.format(prefix))(x)
        
        x = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), 
                             padding='same', name='{}_block3_pool'.format(prefix))(x)
        x = Flatten()(x)
        if include_dense:
            x = Dense(512)(x)
        return x


    XT_SHAPE = (1, 49, 16)
    YT_SHAPE = (1, 49, 15)
    ZT_SHAPE = (1, 49, 18)
    
    
    xt_layer = Input(shape=XT_SHAPE, name='xt_input')
    yt_layer = Input(shape=YT_SHAPE, name='yt_input')
    zt_layer = Input(shape=ZT_SHAPE, name='zt_input')
    
    xt_branch = _conv_block(xt_layer)#, include_dense)
    yt_branch = _conv_block(yt_layer)#, include_dense)
    zt_branch = _conv_block(zt_layer)#, include_dense)
    
    
    # top
    if include_dense:
        x = add([xt_branch, yt_branch, zt_branch], name='merge_add')
    else:
        x = concatenate([xt_branch, yt_branch, zt_branch], name='merge_concat')
        
    x = Dense(256,activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[xt_layer, yt_layer, zt_layer], outputs=predictions, name='vgglike')
    model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [7]:
from multi_files_generator import Xy_gen_XT_YT_ZT

In [8]:
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_sep_model = vgg_separate_channels(**fargs)
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop_noval = EarlyStopping(monitor='loss', patience=5, verbose=1)

fitargs = {'steps_per_epoch' : 10, 'callbacks' : [early_stop_noval, ModelCheckpoint("models/try.hdf5")],
           'epochs': 30, 'verbose': 1}
history_vgg_sep = vgg_sep_model.fit_generator(Xy_gen_XT_YT_ZT(fnames_train, batch_size=64), **fitargs)


Epoch 1/30
file 0 : train_test_files/Xy_train1_sel5_doms.npz
10/10 [==============================] - 10s - loss: 0.6439 - acc: 0.6578    
Epoch 2/30
10/10 [==============================] - 5s - loss: 0.5296 - acc: 0.7359     
Epoch 3/30
 2/10 [=====>........................] - ETA: 4s - loss: 0.3783 - acc: 0.828132
file 1 : train_test_files/Xy_train2_sel5_doms.npz
10/10 [==============================] - 5s - loss: 0.4844 - acc: 0.8141     
Epoch 4/30
10/10 [==============================] - 5s - loss: 0.3912 - acc: 0.8240     
Epoch 5/30
10/10 [==============================] - 5s - loss: 0.3939 - acc: 0.8391     
Epoch 6/30
 5/10 [==============>...............] - ETA: 2s - loss: 0.3659 - acc: 0.831365
file 2 : train_test_files/Xy_train3_sel5_doms.npz
10/10 [==============================] - 6s - loss: 0.3499 - acc: 0.8531     
Epoch 7/30
10/10 [==============================] - 6s - loss: 0.3476 - acc: 0.8512     
Epoch 8/30
10/10 [==============================] - 5s - loss: 0.36

In [8]:
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_sep_model = vgg_separate_channels(**fargs)
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop_noval = EarlyStopping(monitor='loss', patience=5, verbose=1)

fitargs = {'steps_per_epoch' : iter_train, 'callbacks' : [early_stop_noval, ModelCheckpoint("models/vgg_sep_XT_YT_ZT_60files64.hdf5")],
           'epochs': 30, 'verbose': 1}
history_vgg_sep = vgg_sep_model.fit_generator(Xy_gen_XT_YT_ZT(fnames_train, batch_size=64), **fitargs)


Epoch 1/30file 0 : train_test_files/Xy_train1_sel5_doms.npz

  22/1986 [..............................] - ETA: 624s - loss: 0.5708 - acc: 0.725132
file 1 : train_test_files/Xy_train2_sel5_doms.npz
  55/1986 [..............................] - ETA: 443s - loss: 0.4721 - acc: 0.786265
file 2 : train_test_files/Xy_train3_sel5_doms.npz
  89/1986 [>.............................] - ETA: 385s - loss: 0.4169 - acc: 0.818899
file 3 : train_test_files/Xy_train4_sel5_doms.npz
 122/1986 [>.............................] - ETA: 362s - loss: 0.3886 - acc: 0.8345132
file 4 : train_test_files/Xy_train5_sel5_doms.npz
 155/1986 [=>............................] - ETA: 343s - loss: 0.3674 - acc: 0.8449165
file 5 : train_test_files/Xy_train6_sel5_doms.npz
 188/1986 [=>............................] - ETA: 331s - loss: 0.3571 - acc: 0.8504198
file 6 : train_test_files/Xy_train7_sel5_doms.npz
 221/1986 [==>...........................] - ETA: 320s - loss: 0.3409 - acc: 0.8583231
file 7 : train_test_files/Xy_trai

1812/1986 [==========================>...] - ETA: 28s - loss: 0.2074 - acc: 0.92001822
file 55 : train_test_files/Xy_train56_sel5_doms.npz
1845/1986 [==========================>...] - ETA: 23s - loss: 0.2077 - acc: 0.91991855
file 56 : train_test_files/Xy_train57_sel5_doms.npz
1878/1986 [===========================>..] - ETA: 17s - loss: 0.2076 - acc: 0.91991888
file 57 : train_test_files/Xy_train58_sel5_doms.npz
1910/1986 [===========================>..] - ETA: 12s - loss: 0.2066 - acc: 0.92021920
file 58 : train_test_files/Xy_train59_sel5_doms.npz
1943/1986 [============================>.] - ETA: 7s - loss: 0.2059 - acc: 0.9205- ETA: 7s - loss: 0.2060 - acc: 0.1953
file 59 : train_test_files/Xy_train60_sel5_doms.npz
1976/1986 [============================>.] - ETA: 1s - loss: 0.2060 - acc: 0.92051986
file 0 : train_test_files/Xy_train1_sel5_doms.npz
1986/1986 [==============================] - 327s - loss: 0.2060 - acc: 0.9205   
Epoch 3/30
  22/1986 [..............................] 

1645/1986 [=======================>......] - ETA: 56s - loss: 0.1895 - acc: 0.92751655
file 50 : train_test_files/Xy_train51_sel5_doms.npz
1678/1986 [========================>.....] - ETA: 51s - loss: 0.1894 - acc: 0.92761688
file 51 : train_test_files/Xy_train52_sel5_doms.npz
1712/1986 [========================>.....] - ETA: 45s - loss: 0.1889 - acc: 0.92781722
file 52 : train_test_files/Xy_train53_sel5_doms.npz
1745/1986 [=========================>....] - ETA: 39s - loss: 0.1890 - acc: 0.92761755
file 53 : train_test_files/Xy_train54_sel5_doms.npz
1778/1986 [=========================>....] - ETA: 34s - loss: 0.1891 - acc: 0.92751788
file 54 : train_test_files/Xy_train55_sel5_doms.npz
1812/1986 [==========================>...] - ETA: 28s - loss: 0.1884 - acc: 0.92781822
file 55 : train_test_files/Xy_train56_sel5_doms.npz
1845/1986 [==========================>...] - ETA: 23s - loss: 0.1888 - acc: 0.92771855
file 56 : train_test_files/Xy_train57_sel5_doms.npz
1878/1986 [================

1578/1986 [======================>.......] - ETA: 67s - loss: 0.1832 - acc: 0.9300 ETA1588
file 48 : train_test_files/Xy_train49_sel5_doms.npz
1612/1986 [=======================>......] - ETA: 61s - loss: 0.1832 - acc: 0.93001622
file 49 : train_test_files/Xy_train50_sel5_doms.npz
1645/1986 [=======================>......] - ETA: 56s - loss: 0.1834 - acc: 0.92981655
file 50 : train_test_files/Xy_train51_sel5_doms.npz
1678/1986 [========================>.....] - ETA: 50s - loss: 0.1833 - acc: 0.93001688
file 51 : train_test_files/Xy_train52_sel5_doms.npz
1712/1986 [========================>.....] - ETA: 45s - loss: 0.1828 - acc: 0.93021722
file 52 : train_test_files/Xy_train53_sel5_doms.npz
1745/1986 [=========================>....] - ETA: 39s - loss: 0.1828 - acc: 0.93011755
file 53 : train_test_files/Xy_train54_sel5_doms.npz
1778/1986 [=========================>....] - ETA: 34s - loss: 0.1830 - acc: 0.93011788
file 54 : train_test_files/Xy_train55_sel5_doms.npz
1812/1986 [============

1512/1986 [=====================>........] - ETA: 78s - loss: 0.1778 - acc: 0.93201522
file 46 : train_test_files/Xy_train47_sel5_doms.npz
1545/1986 [======================>.......] - ETA: 73s - loss: 0.1781 - acc: 0.93181555
file 47 : train_test_files/Xy_train48_sel5_doms.npz
1578/1986 [======================>.......] - ETA: 67s - loss: 0.1780 - acc: 0.93191588
file 48 : train_test_files/Xy_train49_sel5_doms.npz
1612/1986 [=======================>......] - ETA: 61s - loss: 0.1780 - acc: 0.93201622
file 49 : train_test_files/Xy_train50_sel5_doms.npz
1645/1986 [=======================>......] - ETA: 56s - loss: 0.1782 - acc: 0.93181655
file 50 : train_test_files/Xy_train51_sel5_doms.npz
1678/1986 [========================>.....] - ETA: 50s - loss: 0.1781 - acc: 0.93201688
file 51 : train_test_files/Xy_train52_sel5_doms.npz
1712/1986 [========================>.....] - ETA: 45s - loss: 0.1775 - acc: 0.93221722
file 52 : train_test_files/Xy_train53_sel5_doms.npz
1745/1986 [================

...

  55/1986 [..............................] - ETA: 372s - loss: 0.0379 - acc: 0.982465
file 2 : train_test_files/Xy_train3_sel5_doms.npz
  89/1986 [>.............................] - ETA: 348s - loss: 0.0408 - acc: 0.981299
file 3 : train_test_files/Xy_train4_sel5_doms.npz
 122/1986 [>.............................] - ETA: 333s - loss: 0.0433 - acc: 0.9813132
file 4 : train_test_files/Xy_train5_sel5_doms.npz
 155/1986 [=>............................] - ETA: 325s - loss: 0.0473 - acc: 0.9799165
file 5 : train_test_files/Xy_train6_sel5_doms.npz
 188/1986 [=>............................] - ETA: 316s - loss: 0.0475 - acc: 0.9801198
file 6 : train_test_files/Xy_train7_sel5_doms.npz
 221/1986 [==>...........................] - ETA: 308s - loss: 0.0455 - acc: 0.9814231
file 7 : train_test_files/Xy_train8_sel5_doms.npz
 254/1986 [==>...........................] - ETA: 301s - loss: 0.0440 - acc: 0.9818264
file 8 : train_test_files/Xy_train9_sel5_doms.npz
 288/1986 [===>..........................] 

1910/1986 [===========================>..] - ETA: 12s - loss: 0.0390 - acc: 0.98531920
file 58 : train_test_files/Xy_train59_sel5_doms.npz
1943/1986 [============================>.] - ETA: 7s - loss: 0.0389 - acc: 0.98541953
file 59 : train_test_files/Xy_train60_sel5_doms.npz
1976/1986 [============================>.] - ETA: 1s - loss: 0.0390 - acc: 0.98541986
file 0 : train_test_files/Xy_train1_sel5_doms.npz
1986/1986 [==============================] - 335s - loss: 0.0392 - acc: 0.9853   
Epoch 27/30
  22/1986 [..............................] - ETA: 425s - loss: 0.0297 - acc: 0.991532
file 1 : train_test_files/Xy_train2_sel5_doms.npz
  55/1986 [..............................] - ETA: 367s - loss: 0.0418 - acc: 0.985865
file 2 : train_test_files/Xy_train3_sel5_doms.npz
  89/1986 [>.............................] - ETA: 348s - loss: 0.0400 - acc: 0.9852- ETA: 375s - loss: 0.0425 - ac99
file 3 : train_test_files/Xy_train4_sel5_doms.npz
 122/1986 [>.............................] - ETA: 336s

1845/1986 [==========================>...] - ETA: 24s - loss: 0.0355 - acc: 0.98691855
file 56 : train_test_files/Xy_train57_sel5_doms.npz
1878/1986 [===========================>..] - ETA: 18s - loss: 0.0358 - acc: 0.98691888
file 57 : train_test_files/Xy_train58_sel5_doms.npz
1910/1986 [===========================>..] - ETA: 12s - loss: 0.0359 - acc: 0.98681920
file 58 : train_test_files/Xy_train59_sel5_doms.npz
1943/1986 [============================>.] - ETA: 7s - loss: 0.0357 - acc: 0.98691953
file 59 : train_test_files/Xy_train60_sel5_doms.npz
1976/1986 [============================>.] - ETA: 1s - loss: 0.0357 - acc: 0.9869- ETA: 6s1986
file 0 : train_test_files/Xy_train1_sel5_doms.npz
1986/1986 [==============================] - 337s - loss: 0.0358 - acc: 0.9869   
Epoch 28/30
  22/1986 [..............................] - ETA: 441s - loss: 0.0312 - acc: 0.988632
file 1 : train_test_files/Xy_train2_sel5_doms.npz
  55/1986 [..............................] - ETA: 372s - loss: 0.0338 

1778/1986 [=========================>....] - ETA: 35s - loss: 0.0313 - acc: 0.98851788
file 54 : train_test_files/Xy_train55_sel5_doms.npz
1812/1986 [==========================>...] - ETA: 29s - loss: 0.0314 - acc: 0.98851822
file 55 : train_test_files/Xy_train56_sel5_doms.npz
1845/1986 [==========================>...] - ETA: 24s - loss: 0.0318 - acc: 0.98841855
file 56 : train_test_files/Xy_train57_sel5_doms.npz
1878/1986 [===========================>..] - ETA: 18s - loss: 0.0320 - acc: 0.98831888
file 57 : train_test_files/Xy_train58_sel5_doms.npz
1910/1986 [===========================>..] - ETA: 12s - loss: 0.0318 - acc: 0.98841920
file 58 : train_test_files/Xy_train59_sel5_doms.npz
1943/1986 [============================>.] - ETA: 7s - loss: 0.0317 - acc: 0.98841953
file 59 : train_test_files/Xy_train60_sel5_doms.npz
1976/1986 [============================>.] - ETA: 1s - loss: 0.0317 - acc: 0.98841986
file 0 : train_test_files/Xy_train1_sel5_doms.npz
1986/1986 [====================

1645/1986 [=======================>......] - ETA: 58s - loss: 0.0286 - acc: 0.98951655
file 50 : train_test_files/Xy_train51_sel5_doms.npz
1678/1986 [========================>.....] - ETA: 52s - loss: 0.0285 - acc: 0.98961688
file 51 : train_test_files/Xy_train52_sel5_doms.npz
1712/1986 [========================>.....] - ETA: 46s - loss: 0.0282 - acc: 0.98961722
file 52 : train_test_files/Xy_train53_sel5_doms.npz
1745/1986 [=========================>....] - ETA: 41s - loss: 0.0281 - acc: 0.98971755
file 53 : train_test_files/Xy_train54_sel5_doms.npz
1778/1986 [=========================>....] - ETA: 35s - loss: 0.0281 - acc: 0.98971788
file 54 : train_test_files/Xy_train55_sel5_doms.npz
1812/1986 [==========================>...] - ETA: 29s - loss: 0.0283 - acc: 0.98971822
file 55 : train_test_files/Xy_train56_sel5_doms.npz
1845/1986 [==========================>...] - ETA: 24s - loss: 0.0284 - acc: 0.98961855
file 56 : train_test_files/Xy_train57_sel5_doms.npz
1878/1986 [================

In [15]:
test_evts, iter_test

(31318, 534)

In [25]:
ytest=[]
for i, (X_test, y_test) in enumerate(Xy_gen_XT_YT_ZT(fnames_test)):
    ytest.append(y_test)
    if i==533:
        break

file 0 : train_test_files/Xy_test1_sel5_doms.npz
8
file 1 : train_test_files/Xy_test2_sel5_doms.npz
17
file 2 : train_test_files/Xy_test3_sel5_doms.npz
26
file 3 : train_test_files/Xy_test4_sel5_doms.npz
35
file 4 : train_test_files/Xy_test5_sel5_doms.npz
44
file 5 : train_test_files/Xy_test6_sel5_doms.npz
53
file 6 : train_test_files/Xy_test7_sel5_doms.npz
62
file 7 : train_test_files/Xy_test8_sel5_doms.npz
71
file 8 : train_test_files/Xy_test9_sel5_doms.npz
80
file 9 : train_test_files/Xy_test10_sel5_doms.npz
89
file 10 : train_test_files/Xy_test11_sel5_doms.npz
98
file 11 : train_test_files/Xy_test12_sel5_doms.npz
106
file 12 : train_test_files/Xy_test13_sel5_doms.npz
114
file 13 : train_test_files/Xy_test14_sel5_doms.npz
123
file 14 : train_test_files/Xy_test15_sel5_doms.npz
132
file 15 : train_test_files/Xy_test16_sel5_doms.npz
141
file 16 : train_test_files/Xy_test17_sel5_doms.npz
150
file 17 : train_test_files/Xy_test18_sel5_doms.npz
159
file 18 : train_test_files/Xy_test19_sel5

In [28]:
y_real = np.vstack(np.asarray(ytest))
y_real.shape

(31318, 2)

In [33]:
pred = vgg_sep_model.predict_generator(Xy_gen_XT_YT_ZT(fnames_test),steps = 534)

file 0 : train_test_files/Xy_test1_sel5_doms.npz
8
file 1 : train_test_files/Xy_test2_sel5_doms.npz
17
file 2 : train_test_files/Xy_test3_sel5_doms.npz
26
file 3 : train_test_files/Xy_test4_sel5_doms.npz
35
file 4 : train_test_files/Xy_test5_sel5_doms.npz
44
file 5 : train_test_files/Xy_test6_sel5_doms.npz
53
file 6 : train_test_files/Xy_test7_sel5_doms.npz
62
file 7 : train_test_files/Xy_test8_sel5_doms.npz
71
file 8 : train_test_files/Xy_test9_sel5_doms.npz
80
file 9 : train_test_files/Xy_test10_sel5_doms.npz
89
file 10 : train_test_files/Xy_test11_sel5_doms.npz
98
file 11 : train_test_files/Xy_test12_sel5_doms.npz
106
file 12 : train_test_files/Xy_test13_sel5_doms.npz
114
file 13 : train_test_files/Xy_test14_sel5_doms.npz
123
file 14 : train_test_files/Xy_test15_sel5_doms.npz
132
file 15 : train_test_files/Xy_test16_sel5_doms.npz
141
file 16 : train_test_files/Xy_test17_sel5_doms.npz
150
file 17 : train_test_files/Xy_test18_sel5_doms.npz
159
file 18 : train_test_files/Xy_test19_sel5

In [34]:
ppred = np.vstack(pred)
ppred.shape

(31318, 2)

In [35]:
predicted_y = np.argmax(pred, axis=1)

In [36]:
from sklearn.metrics import accuracy_score
score = accuracy_score(np.argmax(y_real, axis=1), predicted_y)
score

0.9048151222938885

### testing with normalized events

In [8]:
fnames_train = np.asarray(["train_test_files/norm/Xy_train"+str(i+1)+"_sel5_doms_norm.npz" for i in range(0,60)])
fnames_test = np.asarray(["train_test_files/norm/Xy_test"+str(i+1)+"_sel5_doms_norm.npz" for i in range(0,60)])

In [9]:
iter_train, train_evts = get_iterations.get_n_iterations(fnames_train)
print(iter_train, train_evts)
iter_test, test_evts = get_iterations.get_n_iterations(fnames_test)
print(iter_test, test_evts)

(1986, 125153)
(534, 31318)


In [11]:
iter_train

1986

In [10]:
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_sep_model = vgg_separate_channels(**fargs)
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop_noval = EarlyStopping(monitor='loss', patience=5, verbose=1)

fitargs = {'steps_per_epoch' : iter_train, 'callbacks' : [early_stop_noval, ModelCheckpoint("models/vgg_sep_XT_YT_ZT_60files64_norm.hdf5")],
           'epochs': 30, 'verbose': 1}
history_vgg_sep = vgg_sep_model.fit_generator(Xy_gen_XT_YT_ZT(fnames_train, batch_size=64), **fitargs)


Epoch 1/30file 0 : train_test_files/norm/Xy_train1_sel5_doms_norm.npz

  22/1986 [..............................] - ETA: 619s - loss: 0.5632 - acc: 0.728732
file 1 : train_test_files/norm/Xy_train2_sel5_doms_norm.npz
  55/1986 [..............................] - ETA: 439s - loss: 0.4760 - acc: 0.786265
file 2 : train_test_files/norm/Xy_train3_sel5_doms_norm.npz
  89/1986 [>.............................] - ETA: 391s - loss: 0.4240 - acc: 0.815099
file 3 : train_test_files/norm/Xy_train4_sel5_doms_norm.npz
 122/1986 [>.............................] - ETA: 363s - loss: 0.3938 - acc: 0.8311132
file 4 : train_test_files/norm/Xy_train5_sel5_doms_norm.npz
 155/1986 [=>............................] - ETA: 345s - loss: 0.3728 - acc: 0.8415165
file 5 : train_test_files/norm/Xy_train6_sel5_doms_norm.npz
 156/1986 [=>............................] - ETA: 343s - loss: 0.3723 - acc: 0.8418

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 763, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/usr/local/lib/python2.7/dist-packages/keras/utils/data_utils.py", line 568, in data_generator_task
    generator_output = next(self._generator)
  File "multi_files_generator.py", line 112, in Xy_gen_XT_YT_ZT
    X = Xy['x']
  File "/home/cdesio/.local/lib/python2.7/site-packages/numpy/lib/npyio.py", line 226, in __getitem__
    bytes = self.zip.open(key)
  File "/usr/lib/python2.7/zipfile.py", line 971, in open
    raise BadZipfile("Bad magic number for file header")
BadZipfile: Bad magic number for file header



StopIteration: 

In [14]:
Xy =np.load("train_test_files/norm/Xy_train1_sel5_doms_norm.npz")

In [22]:
np.all(Xy["x"]==Xy_nn["x"])

True

In [19]:
Xy_nn = np.load("train_test_files/Xy_train1_sel5_doms.npz")

In [20]:
Xy_nn["x"]

array([386,  40,  13, ...,   9,   7, 101], dtype=uint64)

### try with Zt,Xy

In [23]:
fnames_train = np.asarray(["train_test_files/Xy_train"+str(i+1)+"_sel5_doms"+".npz" for i in range(0,60)])
fnames_test = np.asarray(["train_test_files/Xy_test"+str(i+1)+"_sel5_doms"+".npz" for i in range(0,60)])

In [50]:
from keras.layers import Input
from keras.layers import concatenate, add
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.core import Flatten, Dense

from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

def vgg_separate_channels(num_classes, optimizer, include_dense=False):
    """VGG inspired Convolutional Networks with multiple inputs
    
    Parameters
    ----------
    num_classes : int
        Number of classes to predict
            
    optimizer : keras.optimizers.Optimizer
        Instance of Keras optimizer to attach to the resulting network
        
    include_dense : bool (default False)
        Flag indicating wether to include a Dense Layer at the top 
        of each separate vgg-like sub-network.
    """
    
    def _conv_block(input_layer, include_dense=False):
        """VGG-inspired Convolutional Blocks"""
        
        prefix = input_layer.name.split('_')[0]

        # Block 1
        x = Conv2D(32, kernel_size=(3, 3), activation='relu', 
                   padding='same', name='{}_block1_conv1'.format(prefix))(input_layer)
        
        x = Conv2D(32, kernel_size=(3, 3), activation='relu', 
                   padding='same', name='{}_block1_conv2'.format(prefix))(x)
        
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), 
                         padding='same', name='{}_block1_pool'.format(prefix))(x)

        # Block 2
        x = Conv2D(64, kernel_size=(3, 3), activation='relu', 
                   padding='same', name='{}_block2_conv1'.format(prefix))(x)
        
        x = Conv2D(64, kernel_size=(3, 3), activation='relu', 
                   padding='same', name='{}_block2_conv2'.format(prefix))(x)
        
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), 
                         padding='same', name='{}_block2_pool'.format(prefix))(x)

        # Block 3
        x = Conv2D(128, kernel_size=(3, 3), activation='relu', 
                   padding='same', name='{}_block3_conv2'.format(prefix))(x)
        
        x = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), 
                             padding='same', name='{}_block3_pool'.format(prefix))(x)
        x = Flatten()(x)
        if include_dense:
            x = Dense(512)(x)
        return x


    ZT_SHAPE = (1, 49, 18)
    XY_SHAPE = (1, 16, 15)
    
    zt_layer = Input(shape=ZT_SHAPE, name='zt_input')
    xy_layer = Input(shape=XY_SHAPE, name='xy_input')
    
    zt_branch = _conv_block(zt_layer)#, include_dense)
    xy_branch = _conv_block(xy_layer)#, include_dense)
    
    # top
    if include_dense:
        x = add([zt_branch, xy_branch], name='merge_add')
    else:
        x = concatenate([zt_branch, xy_branch], name='merge_concat')
        
    x = Dense(256,activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[zt_layer, xy_layer], outputs=predictions, name='vgglike')
    model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])
    return model

In [35]:
import multi_files_generator
reload(multi_files_generator)

<module 'multi_files_generator' from 'multi_files_generator.py'>

In [29]:
from multi_files_generator import Xy_gen_ZT_XY

In [36]:
ytrain=[]
for i, (X_test, y_train) in enumerate(multi_files_generator.Xy_gen_ZT_XY(fnames_train)):
    ytrain.append(y_train)
    if i==iter_train:
        break

file 0 : train_test_files/Xy_train1_sel5_doms.npz
32
file 1 : train_test_files/Xy_train2_sel5_doms.npz
65
file 2 : train_test_files/Xy_train3_sel5_doms.npz
99
file 3 : train_test_files/Xy_train4_sel5_doms.npz
132
file 4 : train_test_files/Xy_train5_sel5_doms.npz
165
file 5 : train_test_files/Xy_train6_sel5_doms.npz
198
file 6 : train_test_files/Xy_train7_sel5_doms.npz
231
file 7 : train_test_files/Xy_train8_sel5_doms.npz
264
file 8 : train_test_files/Xy_train9_sel5_doms.npz
298
file 9 : train_test_files/Xy_train10_sel5_doms.npz
331
file 10 : train_test_files/Xy_train11_sel5_doms.npz
365
file 11 : train_test_files/Xy_train12_sel5_doms.npz
396
file 12 : train_test_files/Xy_train13_sel5_doms.npz
428
file 13 : train_test_files/Xy_train14_sel5_doms.npz
461
file 14 : train_test_files/Xy_train15_sel5_doms.npz
494
file 15 : train_test_files/Xy_train16_sel5_doms.npz
527
file 16 : train_test_files/Xy_train17_sel5_doms.npz
560
file 17 : train_test_files/Xy_train18_sel5_doms.npz
593
file 18 : trai

In [38]:
X_test,y_test = next(multi_files_generator.Xy_gen_ZT_XY(fnames_train))

file 0 : train_test_files/Xy_train1_sel5_doms.npz


In [46]:
X_test[0].shape

(64, 1, 49, 18)

In [34]:
np.asarray(ytrain).shape

(1987,)

In [51]:
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_sep_model = vgg_separate_channels(**fargs)
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop_noval = EarlyStopping(monitor='loss', patience=5, verbose=1)

fitargs = {'steps_per_epoch' : iter_train, 'callbacks' : [early_stop_noval, ModelCheckpoint("models/vgg_sep_ZT_XY_60files64.hdf5")],
           'epochs': 30, 'verbose': 1}
history_vgg_sep = vgg_sep_model.fit_generator(multi_files_generator.Xy_gen_ZT_XY(fnames_train, batch_size=64), **fitargs)


Epoch 1/30
file 0 : train_test_files/Xy_train1_sel5_doms.npz
  21/1986 [..............................] - ETA: 402s - loss: 0.6513 - acc: 0.642932
file 1 : train_test_files/Xy_train2_sel5_doms.npz
  55/1986 [..............................] - ETA: 292s - loss: 0.5142 - acc: 0.749065
file 2 : train_test_files/Xy_train3_sel5_doms.npz
  88/1986 [>.............................] - ETA: 266s - loss: 0.4545 - acc: 0.788499
file 3 : train_test_files/Xy_train4_sel5_doms.npz
 122/1986 [>.............................] - ETA: 247s - loss: 0.4176 - acc: 0.8110132
file 4 : train_test_files/Xy_train5_sel5_doms.npz
 155/1986 [=>............................] - ETA: 238s - loss: 0.3923 - acc: 0.8238165
file 5 : train_test_files/Xy_train6_sel5_doms.npz
 188/1986 [=>............................] - ETA: 231s - loss: 0.3786 - acc: 0.8326198
file 6 : train_test_files/Xy_train7_sel5_doms.npz
 221/1986 [==>...........................] - ETA: 225s - loss: 0.3603 - acc: 0.8423231
file 7 : train_test_files/Xy_trai

1976/1986 [============================>.] - ETA: 1s - loss: 0.2408 - acc: 0.90591986
file 0 : train_test_files/Xy_train1_sel5_doms.npz
1986/1986 [==============================] - 239s - loss: 0.2408 - acc: 0.9058   
Epoch 2/30
  21/1986 [..............................] - ETA: 287s - loss: 0.1998 - acc: 0.924932
file 1 : train_test_files/Xy_train2_sel5_doms.npz
  55/1986 [..............................] - ETA: 250s - loss: 0.2085 - acc: 0.921865
file 2 : train_test_files/Xy_train3_sel5_doms.npz
  88/1986 [>.............................] - ETA: 240s - loss: 0.2132 - acc: 0.919799
file 3 : train_test_files/Xy_train4_sel5_doms.npz
 122/1986 [>.............................] - ETA: 232s - loss: 0.2138 - acc: 0.9187132
file 4 : train_test_files/Xy_train5_sel5_doms.npz
 155/1986 [=>............................] - ETA: 227s - loss: 0.2114 - acc: 0.9203165
file 5 : train_test_files/Xy_train6_sel5_doms.npz
 188/1986 [=>............................] - ETA: 222s - loss: 0.2126 - acc: 0.9194198
fi

1845/1986 [==========================>...] - ETA: 17s - loss: 0.1992 - acc: 0.92361855
file 56 : train_test_files/Xy_train57_sel5_doms.npz
1878/1986 [===========================>..] - ETA: 13s - loss: 0.1992 - acc: 0.92351888
file 57 : train_test_files/Xy_train58_sel5_doms.npz
1909/1986 [===========================>..] - ETA: 9s - loss: 0.1985 - acc: 0.92371920
file 58 : train_test_files/Xy_train59_sel5_doms.npz
1943/1986 [============================>.] - ETA: 5s - loss: 0.1978 - acc: 0.92391953
file 59 : train_test_files/Xy_train60_sel5_doms.npz
1976/1986 [============================>.] - ETA: 1s - loss: 0.1979 - acc: 0.92391986
file 0 : train_test_files/Xy_train1_sel5_doms.npz
1986/1986 [==============================] - 245s - loss: 0.1980 - acc: 0.9238   
Epoch 4/30
  21/1986 [..............................] - ETA: 337s - loss: 0.1821 - acc: 0.933832
file 1 : train_test_files/Xy_train2_sel5_doms.npz
  55/1986 [..............................] - ETA: 271s - loss: 0.1904 - acc: 0.92

1711/1986 [========================>.....] - ETA: 33s - loss: 0.1902 - acc: 0.92721722
file 52 : train_test_files/Xy_train53_sel5_doms.npz
1745/1986 [=========================>....] - ETA: 29s - loss: 0.1903 - acc: 0.92711755
file 53 : train_test_files/Xy_train54_sel5_doms.npz
1778/1986 [=========================>....] - ETA: 25s - loss: 0.1903 - acc: 0.92701788
file 54 : train_test_files/Xy_train55_sel5_doms.npz
1811/1986 [==========================>...] - ETA: 21s - loss: 0.1896 - acc: 0.92731822
file 55 : train_test_files/Xy_train56_sel5_doms.npz
1845/1986 [==========================>...] - ETA: 17s - loss: 0.1898 - acc: 0.92731855
file 56 : train_test_files/Xy_train57_sel5_doms.npz
1878/1986 [===========================>..] - ETA: 13s - loss: 0.1898 - acc: 0.92731888
file 57 : train_test_files/Xy_train58_sel5_doms.npz
1909/1986 [===========================>..] - ETA: 9s - loss: 0.1891 - acc: 0.92751920
file 58 : train_test_files/Xy_train59_sel5_doms.npz
1943/1986 [=================

1578/1986 [======================>.......] - ETA: 49s - loss: 0.1832 - acc: 0.93031588
file 48 : train_test_files/Xy_train49_sel5_doms.npz
1611/1986 [=======================>......] - ETA: 45s - loss: 0.1833 - acc: 0.93031622
file 49 : train_test_files/Xy_train50_sel5_doms.npz
1645/1986 [=======================>......] - ETA: 41s - loss: 0.1834 - acc: 0.93031655
file 50 : train_test_files/Xy_train51_sel5_doms.npz
1678/1986 [========================>.....] - ETA: 37s - loss: 0.1832 - acc: 0.93041688
file 51 : train_test_files/Xy_train52_sel5_doms.npz
1711/1986 [========================>.....] - ETA: 33s - loss: 0.1827 - acc: 0.93071722
file 52 : train_test_files/Xy_train53_sel5_doms.npz
1745/1986 [=========================>....] - ETA: 29s - loss: 0.1827 - acc: 0.93061755
file 53 : train_test_files/Xy_train54_sel5_doms.npz
1778/1986 [=========================>....] - ETA: 25s - loss: 0.1827 - acc: 0.93051788
file 54 : train_test_files/Xy_train55_sel5_doms.npz
1811/1986 [================

...

 517/1986 [======>.......................] - ETA: 181s - loss: 0.0916 - acc: 0.9642527
file 16 : train_test_files/Xy_train17_sel5_doms.npz
 550/1986 [=======>......................] - ETA: 176s - loss: 0.0917 - acc: 0.9643560
file 17 : train_test_files/Xy_train18_sel5_doms.npz
 583/1986 [=======>......................] - ETA: 171s - loss: 0.0914 - acc: 0.9645593
file 18 : train_test_files/Xy_train19_sel5_doms.npz
 616/1986 [========>.....................] - ETA: 167s - loss: 0.0912 - acc: 0.9646626
file 19 : train_test_files/Xy_train20_sel5_doms.npz
 649/1986 [========>.....................] - ETA: 163s - loss: 0.0923 - acc: 0.9642659
file 20 : train_test_files/Xy_train21_sel5_doms.npz
 680/1986 [=========>....................] - ETA: 159s - loss: 0.0928 - acc: 0.9638691
file 21 : train_test_files/Xy_train22_sel5_doms.npz
 714/1986 [=========>....................] - ETA: 155s - loss: 0.0921 - acc: 0.9642724
file 22 : train_test_files/Xy_train23_sel5_doms.npz
 747/1986 [==========>.....

 386/1986 [====>.........................] - ETA: 194s - loss: 0.0811 - acc: 0.9688396
file 12 : train_test_files/Xy_train13_sel5_doms.npz
 417/1986 [=====>........................] - ETA: 190s - loss: 0.0816 - acc: 0.9687428
file 13 : train_test_files/Xy_train14_sel5_doms.npz
 451/1986 [=====>........................] - ETA: 185s - loss: 0.0816 - acc: 0.9684461
file 14 : train_test_files/Xy_train15_sel5_doms.npz
 484/1986 [======>.......................] - ETA: 181s - loss: 0.0814 - acc: 0.9685494
file 15 : train_test_files/Xy_train16_sel5_doms.npz
 517/1986 [======>.......................] - ETA: 177s - loss: 0.0824 - acc: 0.9679527
file 16 : train_test_files/Xy_train17_sel5_doms.npz
 550/1986 [=======>......................] - ETA: 173s - loss: 0.0823 - acc: 0.9680560
file 17 : train_test_files/Xy_train18_sel5_doms.npz
 583/1986 [=======>......................] - ETA: 169s - loss: 0.0818 - acc: 0.9680593
file 18 : train_test_files/Xy_train19_sel5_doms.npz
 616/1986 [========>.......

 254/1986 [==>...........................] - ETA: 213s - loss: 0.0704 - acc: 0.9714264
file 8 : train_test_files/Xy_train9_sel5_doms.npz
 287/1986 [===>..........................] - ETA: 210s - loss: 0.0696 - acc: 0.9717298
file 9 : train_test_files/Xy_train10_sel5_doms.npz
 321/1986 [===>..........................] - ETA: 204s - loss: 0.0702 - acc: 0.9714331
file 10 : train_test_files/Xy_train11_sel5_doms.npz
 354/1986 [====>.........................] - ETA: 199s - loss: 0.0718 - acc: 0.9709365
file 11 : train_test_files/Xy_train12_sel5_doms.npz
 386/1986 [====>.........................] - ETA: 195s - loss: 0.0717 - acc: 0.9711396
file 12 : train_test_files/Xy_train13_sel5_doms.npz
 417/1986 [=====>........................] - ETA: 192s - loss: 0.0719 - acc: 0.9711428
file 13 : train_test_files/Xy_train14_sel5_doms.npz
 451/1986 [=====>........................] - ETA: 187s - loss: 0.0717 - acc: 0.9713461
file 14 : train_test_files/Xy_train15_sel5_doms.npz
 484/1986 [======>............

 122/1986 [>.............................] - ETA: 241s - loss: 0.0591 - acc: 0.9753132
file 4 : train_test_files/Xy_train5_sel5_doms.npz
 155/1986 [=>............................] - ETA: 234s - loss: 0.0623 - acc: 0.9746165
file 5 : train_test_files/Xy_train6_sel5_doms.npz
 188/1986 [=>............................] - ETA: 227s - loss: 0.0640 - acc: 0.9734198
file 6 : train_test_files/Xy_train7_sel5_doms.npz
 221/1986 [==>...........................] - ETA: 222s - loss: 0.0609 - acc: 0.9747231
file 7 : train_test_files/Xy_train8_sel5_doms.npz
 254/1986 [==>...........................] - ETA: 216s - loss: 0.0606 - acc: 0.9747264
file 8 : train_test_files/Xy_train9_sel5_doms.npz
 287/1986 [===>..........................] - ETA: 211s - loss: 0.0600 - acc: 0.9750298
file 9 : train_test_files/Xy_train10_sel5_doms.npz
 321/1986 [===>..........................] - ETA: 206s - loss: 0.0608 - acc: 0.9745331
file 10 : train_test_files/Xy_train11_sel5_doms.npz
 354/1986 [====>......................

1986/1986 [==============================] - 237s - loss: 0.0570 - acc: 0.9781   


In [54]:
ytest=[]
for i, (X_test, y_test) in enumerate(multi_files_generator.Xy_gen_ZT_XY(fnames_test)):
    ytest.append(y_test)
    if i==533:
        break

y_real = np.vstack(np.asarray(ytest))
y_real.shape

file 0 : train_test_files/Xy_test1_sel5_doms.npz
8
file 1 : train_test_files/Xy_test2_sel5_doms.npz
17
file 2 : train_test_files/Xy_test3_sel5_doms.npz
26
file 3 : train_test_files/Xy_test4_sel5_doms.npz
35
file 4 : train_test_files/Xy_test5_sel5_doms.npz
44
file 5 : train_test_files/Xy_test6_sel5_doms.npz
53
file 6 : train_test_files/Xy_test7_sel5_doms.npz
62
file 7 : train_test_files/Xy_test8_sel5_doms.npz
71
file 8 : train_test_files/Xy_test9_sel5_doms.npz
80
file 9 : train_test_files/Xy_test10_sel5_doms.npz
89
file 10 : train_test_files/Xy_test11_sel5_doms.npz
98
file 11 : train_test_files/Xy_test12_sel5_doms.npz
106
file 12 : train_test_files/Xy_test13_sel5_doms.npz
114
file 13 : train_test_files/Xy_test14_sel5_doms.npz
123
file 14 : train_test_files/Xy_test15_sel5_doms.npz
132
file 15 : train_test_files/Xy_test16_sel5_doms.npz
141
file 16 : train_test_files/Xy_test17_sel5_doms.npz
150
file 17 : train_test_files/Xy_test18_sel5_doms.npz
159
file 18 : train_test_files/Xy_test19_sel5

(31318, 2)

In [55]:
pred = vgg_sep_model.predict_generator(multi_files_generator.Xy_gen_ZT_XY(fnames_test),steps = 534)


ppred = np.vstack(pred)
ppred.shape

file 0 : train_test_files/Xy_test1_sel5_doms.npz
8
file 1 : train_test_files/Xy_test2_sel5_doms.npz
17
file 2 : train_test_files/Xy_test3_sel5_doms.npz
26
file 3 : train_test_files/Xy_test4_sel5_doms.npz
35
file 4 : train_test_files/Xy_test5_sel5_doms.npz
44
file 5 : train_test_files/Xy_test6_sel5_doms.npz
53
file 6 : train_test_files/Xy_test7_sel5_doms.npz
62
file 7 : train_test_files/Xy_test8_sel5_doms.npz
71
file 8 : train_test_files/Xy_test9_sel5_doms.npz
80
file 9 : train_test_files/Xy_test10_sel5_doms.npz
89
file 10 : train_test_files/Xy_test11_sel5_doms.npz
98
file 11 : train_test_files/Xy_test12_sel5_doms.npz
106
file 12 : train_test_files/Xy_test13_sel5_doms.npz
114
file 13 : train_test_files/Xy_test14_sel5_doms.npz
123
file 14 : train_test_files/Xy_test15_sel5_doms.npz
132
file 15 : train_test_files/Xy_test16_sel5_doms.npz
141
file 16 : train_test_files/Xy_test17_sel5_doms.npz
150
file 17 : train_test_files/Xy_test18_sel5_doms.npz
159
file 18 : train_test_files/Xy_test19_sel5

(31318, 2)

In [56]:
predicted_y = np.argmax(pred, axis=1)

In [57]:

from sklearn.metrics import accuracy_score
score = accuracy_score(np.argmax(y_real, axis=1), predicted_y)
score


0.90494284437064942

In [3]:
from export_train_test import export_train_test

In [23]:
fnames_numu =[]
fnames_nue = []
for i in range(100):
    fnames_numu.append("aafiles/Jpp_v6_aa_files/Xy_files/Xy_numu_"+str(i+1)+"_sel5_doms.npz")
    fnames_nue.append("aafiles/Jpp_v6_aa_files/Xy_files/Xy_nue_"+str(i+1)+"_sel5_doms.npz")

In [6]:
fnames_numu = np.asarray(fnames_numu)
fnames_nue = np.asarray(fnames_nue)


In [50]:
export_train_test?

In [ ]:
for i in range(100):
    export_train_test(i ,fname_numu=fnames_numu[i], fname_nue = fnames_nue[i])

In [56]:
export_train_test(10 ,fname_numu=fnames_numu[10], fname_nue = fnames_nue[10])

In [8]:
fnames_train = ["train_test_files/Xy_train"+str(i+1)+"_sel5_doms"+".npz" for i in range(100)]
fnames_test = ["train_test_files/Xy_test"+str(i+1)+"_sel5_doms"+".npz" for i in range(100)]

In [9]:
from get_iterations import get_n_iterations

In [17]:
get_n_iterations(fnames_train[:10], batch_size=64)

(328, 20688)

In [16]:
get_n_iterations(fnames_test[:10], batch_size=64)

(87, 5179)

In [12]:
from keras.layers import Input
from keras.layers import concatenate, add
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.core import Flatten, Dense

from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

def vgg_separate_channels(num_classes, optimizer, include_dense=False):
    """VGG inspired Convolutional Networks with multiple inputs
    
    Parameters
    ----------
    num_classes : int
        Number of classes to predict
            
    optimizer : keras.optimizers.Optimizer
        Instance of Keras optimizer to attach to the resulting network
        
    include_dense : bool (default False)
        Flag indicating wether to include a Dense Layer at the top 
        of each separate vgg-like sub-network.
    """
    
    def _conv_block(input_layer, include_dense=False):
        """VGG-inspired Convolutional Blocks"""
        
        prefix = input_layer.name.split('_')[0]

        # Block 1
        x = Conv2D(32, kernel_size=(4, 4), activation='relu', 
                   padding='same', name='{}_block1_conv1'.format(prefix))(input_layer)
        
        x = Conv2D(32, kernel_size=(4, 4), activation='relu', 
                   padding='same', name='{}_block1_conv2'.format(prefix))(x)
        
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), 
                         padding='same', name='{}_block1_pool'.format(prefix))(x)

        # Block 2
        x = Conv2D(64, kernel_size=(4, 4), activation='relu', 
                   padding='same', name='{}_block2_conv1'.format(prefix))(x)
        
        x = Conv2D(64, kernel_size=(4, 4), activation='relu', 
                   padding='same', name='{}_block2_conv2'.format(prefix))(x)
        
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), 
                         padding='same', name='{}_block2_pool'.format(prefix))(x)

        # Block 3
        x = Conv2D(128, kernel_size=(4, 4), activation='relu', 
                   padding='same', name='{}_block3_conv2'.format(prefix))(x)
        
        x = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), 
                             padding='same', name='{}_block3_pool'.format(prefix))(x)
        x = Flatten()(x)
        if include_dense:
            x = Dense(512)(x)
        return x


    ZT_SHAPE = (1, 75, 18)
    XY_SHAPE = (1, 16, 15)
    
    zt_layer = Input(shape=ZT_SHAPE, name='zt_input')
    xy_layer = Input(shape=XY_SHAPE, name='xy_input')
    
    zt_branch = _conv_block(zt_layer, include_dense)
    xy_branch = _conv_block(xy_layer, include_dense)
    
    # top
    if include_dense:
        x = add([zt_branch, xy_branch], name='merge_add')
    else:
        x = concatenate([zt_branch, xy_branch], name='merge_concat')
        
    x = Dense(256,activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[zt_layer, xy_layer], outputs=predictions, name='vgglike')
    model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [15]:
from multi_files_generator import Xy_gen_ZT_XY
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop_noval = EarlyStopping(monitor='loss', patience=5, verbose=1)

In [10]:
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_sep_model = vgg_separate_channels(**fargs)

fitargs = {'steps_per_epoch' : 328, 'callbacks' : [early_stop_noval, ModelCheckpoint("vgg_sep_ZT_XY_10files.hdf5")],
           'epochs': 30, 'verbose': 1}
history_vgg_sep = vgg_sep_model.fit_generator(Xy_gen_ZT_XY(fnames_train, batch_size=64), **fitargs)


file 0 : train_test_files/Xy_train1_sel5_doms.npzEpoch 1/30

 21/328 [>.............................] - ETA: 107s - loss: 0.5961 - acc: 0.702432
file 1 : train_test_files/Xy_train2_sel5_doms.npz
 55/328 [====>.........................] - ETA: 63s - loss: 0.4685 - acc: 0.787465
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 49s - loss: 0.4252 - acc: 0.811998
file 3 : train_test_files/Xy_train4_sel5_doms.npz
119/328 [=========>....................] - ETA: 41s - loss: 0.4032 - acc: 0.8259130
file 4 : train_test_files/Xy_train5_sel5_doms.npz
153/328 [============>.................] - ETA: 32s - loss: 0.3773 - acc: 0.8383163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
186/328 [================>.............] - ETA: 25s - loss: 0.3645 - acc: 0.8444196
file 6 : train_test_files/Xy_train7_sel5_doms.npz
217/328 [==================>...........] - ETA: 20s - loss: 0.3501 - acc: 0.8519228
file 7 : train_test_files/Xy_train8_sel5_doms.npz
251

251/328 [=====================>........] - ETA: 13s - loss: 0.2076 - acc: 0.9213261
file 8 : train_test_files/Xy_train9_sel5_doms.npz
284/328 [========================>.....] - ETA: 7s - loss: 0.2083 - acc: 0.9217295
file 9 : train_test_files/Xy_train10_sel5_doms.npz
318/328 [============================>.] - ETA: 1s - loss: 0.2073 - acc: 0.9220328
file 0 : train_test_files/Xy_train1_sel5_doms.npz
328/328 [==============================] - 54s - loss: 0.2070 - acc: 0.9222    
Epoch 7/30
 21/328 [>.............................] - ETA: 74s - loss: 0.1908 - acc: 0.927832
file 1 : train_test_files/Xy_train2_sel5_doms.npz
 55/328 [====>.........................] - ETA: 52s - loss: 0.1961 - acc: 0.924465
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 43s - loss: 0.2039 - acc: 0.921698
file 3 : train_test_files/Xy_train4_sel5_doms.npz
119/328 [=========>....................] - ETA: 37s - loss: 0.2094 - acc: 0.9202130
file 4 : train_test_files

153/328 [============>.................] - ETA: 31s - loss: 0.1900 - acc: 0.9278163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
186/328 [================>.............] - ETA: 25s - loss: 0.1880 - acc: 0.9287196
file 6 : train_test_files/Xy_train7_sel5_doms.npz
217/328 [==================>...........] - ETA: 19s - loss: 0.1862 - acc: 0.9290228
file 7 : train_test_files/Xy_train8_sel5_doms.npz
251/328 [=====================>........] - ETA: 13s - loss: 0.1857 - acc: 0.9291261
file 8 : train_test_files/Xy_train9_sel5_doms.npz
284/328 [========================>.....] - ETA: 7s - loss: 0.1860 - acc: 0.9297295
file 9 : train_test_files/Xy_train10_sel5_doms.npz
318/328 [============================>.] - ETA: 1s - loss: 0.1849 - acc: 0.9304328
file 0 : train_test_files/Xy_train1_sel5_doms.npz
328/328 [==============================] - 55s - loss: 0.1844 - acc: 0.9303    
Epoch 13/30
 21/328 [>.............................] - ETA: 73s - loss: 0.1727 - acc: 0.936832
file 1 : train_test_fi

 55/328 [====>.........................] - ETA: 53s - loss: 0.1547 - acc: 0.941165
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 43s - loss: 0.1588 - acc: 0.940998
file 3 : train_test_files/Xy_train4_sel5_doms.npz
119/328 [=========>....................] - ETA: 37s - loss: 0.1627 - acc: 0.9383130
file 4 : train_test_files/Xy_train5_sel5_doms.npz
153/328 [============>.................] - ETA: 30s - loss: 0.1633 - acc: 0.9382163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
186/328 [================>.............] - ETA: 24s - loss: 0.1609 - acc: 0.9394196
file 6 : train_test_files/Xy_train7_sel5_doms.npz
217/328 [==================>...........] - ETA: 19s - loss: 0.1602 - acc: 0.9391228
file 7 : train_test_files/Xy_train8_sel5_doms.npz
251/328 [=====================>........] - ETA: 13s - loss: 0.1605 - acc: 0.9387261
file 8 : train_test_files/Xy_train9_sel5_doms.npz
284/328 [========================>.....] - ETA: 7s - loss: 0.160

318/328 [============================>.] - ETA: 1s - loss: 0.1265 - acc: 0.9538328
file 0 : train_test_files/Xy_train1_sel5_doms.npz
328/328 [==============================] - 53s - loss: 0.1255 - acc: 0.9542    
Epoch 24/30
 21/328 [>.............................] - ETA: 71s - loss: 0.1191 - acc: 0.958332
file 1 : train_test_files/Xy_train2_sel5_doms.npz
 55/328 [====>.........................] - ETA: 52s - loss: 0.1163 - acc: 0.959565
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 43s - loss: 0.1191 - acc: 0.958498
file 3 : train_test_files/Xy_train4_sel5_doms.npz
119/328 [=========>....................] - ETA: 37s - loss: 0.1201 - acc: 0.9573130
file 4 : train_test_files/Xy_train5_sel5_doms.npz
153/328 [============>.................] - ETA: 30s - loss: 0.1237 - acc: 0.9551163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
186/328 [================>.............] - ETA: 24s - loss: 0.1211 - acc: 0.9564196
file 6 : train_test_file

217/328 [==================>...........] - ETA: 19s - loss: 0.0751 - acc: 0.9756228
file 7 : train_test_files/Xy_train8_sel5_doms.npz
251/328 [=====================>........] - ETA: 13s - loss: 0.0773 - acc: 0.9750261
file 8 : train_test_files/Xy_train9_sel5_doms.npz
284/328 [========================>.....] - ETA: 7s - loss: 0.0762 - acc: 0.9753295
file 9 : train_test_files/Xy_train10_sel5_doms.npz
318/328 [============================>.] - ETA: 1s - loss: 0.0737 - acc: 0.9763328
file 0 : train_test_files/Xy_train1_sel5_doms.npz
328/328 [==============================] - 54s - loss: 0.0732 - acc: 0.9764    
Epoch 30/30
 21/328 [>.............................] - ETA: 70s - loss: 0.0641 - acc: 0.979932
file 1 : train_test_files/Xy_train2_sel5_doms.npz
 55/328 [====>.........................] - ETA: 51s - loss: 0.0717 - acc: 0.974365
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 43s - loss: 0.0693 - acc: 0.976898
file 3 : train_test_file

In [34]:
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_sep_model = vgg_separate_channels(**fargs)

fitargs = {'steps_per_epoch' : 328, 'callbacks' : [early_stop_noval, ModelCheckpoint("vgg_sep_ZT_XY_4x4_10files.hdf5")],
           'epochs': 30, 'verbose': 1}
history_vgg_sep = vgg_sep_model.fit_generator(Xy_gen_ZT_XY(fnames_train, batch_size=64), **fitargs)


file 0 : train_test_files/Xy_train1_sel5_doms.npzEpoch 1/30

 21/328 [>.............................] - ETA: 82s - loss: 0.6042 - acc: 0.696432
file 1 : train_test_files/Xy_train2_sel5_doms.npz
 55/328 [====>.........................] - ETA: 55s - loss: 0.4911 - acc: 0.774965
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 45s - loss: 0.4372 - acc: 0.807398
file 3 : train_test_files/Xy_train4_sel5_doms.npz
119/328 [=========>....................] - ETA: 38s - loss: 0.4114 - acc: 0.8240130
file 4 : train_test_files/Xy_train5_sel5_doms.npz
153/328 [============>.................] - ETA: 31s - loss: 0.3816 - acc: 0.8383163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
186/328 [================>.............] - ETA: 25s - loss: 0.3679 - acc: 0.8455196
file 6 : train_test_files/Xy_train7_sel5_doms.npz
217/328 [==================>...........] - ETA: 19s - loss: 0.3520 - acc: 0.8534228
file 7 : train_test_files/Xy_train8_sel5_doms.npz
251/

251/328 [=====================>........] - ETA: 13s - loss: 0.2037 - acc: 0.9239261
file 8 : train_test_files/Xy_train9_sel5_doms.npz
284/328 [========================>.....] - ETA: 7s - loss: 0.2043 - acc: 0.9244295
file 9 : train_test_files/Xy_train10_sel5_doms.npz
318/328 [============================>.] - ETA: 1s - loss: 0.2034 - acc: 0.9249328
file 0 : train_test_files/Xy_train1_sel5_doms.npz
328/328 [==============================] - 55s - loss: 0.2030 - acc: 0.9251    
Epoch 7/30
 21/328 [>.............................] - ETA: 72s - loss: 0.1843 - acc: 0.935332
file 1 : train_test_files/Xy_train2_sel5_doms.npz
 55/328 [====>.........................] - ETA: 51s - loss: 0.1927 - acc: 0.928965
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 43s - loss: 0.2004 - acc: 0.924398
file 3 : train_test_files/Xy_train4_sel5_doms.npz
119/328 [=========>....................] - ETA: 37s - loss: 0.2069 - acc: 0.9218130
file 4 : train_test_files

153/328 [============>.................] - ETA: 30s - loss: 0.1839 - acc: 0.9281163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
186/328 [================>.............] - ETA: 24s - loss: 0.1818 - acc: 0.9305196
file 6 : train_test_files/Xy_train7_sel5_doms.npz
217/328 [==================>...........] - ETA: 19s - loss: 0.1803 - acc: 0.9317228
file 7 : train_test_files/Xy_train8_sel5_doms.npz
251/328 [=====================>........] - ETA: 13s - loss: 0.1795 - acc: 0.9319261
file 8 : train_test_files/Xy_train9_sel5_doms.npz
284/328 [========================>.....] - ETA: 7s - loss: 0.1793 - acc: 0.9325295
file 9 : train_test_files/Xy_train10_sel5_doms.npz
318/328 [============================>.] - ETA: 1s - loss: 0.1781 - acc: 0.9331328
file 0 : train_test_files/Xy_train1_sel5_doms.npz
328/328 [==============================] - 54s - loss: 0.1776 - acc: 0.9333    
Epoch 13/30
 21/328 [>.............................] - ETA: 71s - loss: 0.1659 - acc: 0.938232
file 1 : train_test_fi

 55/328 [====>.........................] - ETA: 56s - loss: 0.1433 - acc: 0.947665
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 46s - loss: 0.1496 - acc: 0.941798
file 3 : train_test_files/Xy_train4_sel5_doms.npz
119/328 [=========>....................] - ETA: 39s - loss: 0.1523 - acc: 0.9411130
file 4 : train_test_files/Xy_train5_sel5_doms.npz
153/328 [============>.................] - ETA: 31s - loss: 0.1506 - acc: 0.9419163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
186/328 [================>.............] - ETA: 25s - loss: 0.1487 - acc: 0.9441196
file 6 : train_test_files/Xy_train7_sel5_doms.npz
217/328 [==================>...........] - ETA: 19s - loss: 0.1475 - acc: 0.9456228
file 7 : train_test_files/Xy_train8_sel5_doms.npz
251/328 [=====================>........] - ETA: 13s - loss: 0.1469 - acc: 0.9458261
file 8 : train_test_files/Xy_train9_sel5_doms.npz
284/328 [========================>.....] - ETA: 7s - loss: 0.146

318/328 [============================>.] - ETA: 1s - loss: 0.1016 - acc: 0.9645328
file 0 : train_test_files/Xy_train1_sel5_doms.npz
328/328 [==============================] - 55s - loss: 0.1012 - acc: 0.9647    
Epoch 24/30
 21/328 [>.............................] - ETA: 73s - loss: 0.0969 - acc: 0.968032
file 1 : train_test_files/Xy_train2_sel5_doms.npz
 55/328 [====>.........................] - ETA: 52s - loss: 0.0974 - acc: 0.964165
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 44s - loss: 0.0988 - acc: 0.964098
file 3 : train_test_files/Xy_train4_sel5_doms.npz
119/328 [=========>....................] - ETA: 38s - loss: 0.1000 - acc: 0.9642130
file 4 : train_test_files/Xy_train5_sel5_doms.npz
153/328 [============>.................] - ETA: 31s - loss: 0.0990 - acc: 0.9645163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
186/328 [================>.............] - ETA: 24s - loss: 0.0968 - acc: 0.9657196
file 6 : train_test_file

217/328 [==================>...........] - ETA: 19s - loss: 0.0580 - acc: 0.9802228
file 7 : train_test_files/Xy_train8_sel5_doms.npz
251/328 [=====================>........] - ETA: 13s - loss: 0.0571 - acc: 0.9803261
file 8 : train_test_files/Xy_train9_sel5_doms.npz
284/328 [========================>.....] - ETA: 7s - loss: 0.0551 - acc: 0.9808295
file 9 : train_test_files/Xy_train10_sel5_doms.npz
318/328 [============================>.] - ETA: 1s - loss: 0.0534 - acc: 0.9818328
file 0 : train_test_files/Xy_train1_sel5_doms.npz
328/328 [==============================] - 54s - loss: 0.0527 - acc: 0.9821    
Epoch 30/30
 21/328 [>.............................] - ETA: 74s - loss: 0.0701 - acc: 0.972532
file 1 : train_test_files/Xy_train2_sel5_doms.npz
 55/328 [====>.........................] - ETA: 52s - loss: 0.0559 - acc: 0.978965
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 43s - loss: 0.0575 - acc: 0.977998
file 3 : train_test_file

In [62]:
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_sep_model = vgg_separate_channels(**fargs)

fitargs = {'steps_per_epoch' : 3288, 'callbacks' : [early_stop_noval, ModelCheckpoint("vgg_sep_ZT_XY_100files.hdf5")],
           'epochs': 30, 'verbose': 1}
history_vgg_sep = vgg_sep_model.fit_generator(Xy_gen_ZT_XY(fnames_train, batch_size=64), **fitargs)


Epoch 1/30
file 0 : train_test_files/Xy_train1_sel5_doms.npz
  21/3288 [..............................] - ETA: 866s - loss: 0.5957 - acc: 0.683832
file 1 : train_test_files/Xy_train2_sel5_doms.npz
  55/3288 [..............................] - ETA: 657s - loss: 0.4787 - acc: 0.772565
file 2 : train_test_files/Xy_train3_sel5_doms.npz
  88/3288 [..............................] - ETA: 611s - loss: 0.4272 - acc: 0.805198
file 3 : train_test_files/Xy_train4_sel5_doms.npz
 119/3288 [>.............................] - ETA: 587s - loss: 0.4033 - acc: 0.8222130
file 4 : train_test_files/Xy_train5_sel5_doms.npz
 153/3288 [>.............................] - ETA: 561s - loss: 0.3753 - acc: 0.8366163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
 186/3288 [>.............................] - ETA: 548s - loss: 0.3619 - acc: 0.8441196
file 6 : train_test_files/Xy_train7_sel5_doms.npz
 217/3288 [>.............................] - ETA: 538s - loss: 0.3468 - acc: 0.8524228
file 7 : train_test_files/Xy_trai

1929/3288 [================>.............] - ETA: 227s - loss: 0.2379 - acc: 0.90831939
file 59 : train_test_files/Xy_train60_sel5_doms.npz
1960/3288 [================>.............] - ETA: 222s - loss: 0.2375 - acc: 0.90851971
file 60 : train_test_files/Xy_train61_sel5_doms.npz
1994/3288 [=================>............] - ETA: 216s - loss: 0.2367 - acc: 0.90872004
file 61 : train_test_files/Xy_train62_sel5_doms.npz
2027/3288 [=================>............] - ETA: 210s - loss: 0.2362 - acc: 0.90892038
file 62 : train_test_files/Xy_train63_sel5_doms.npz
2061/3288 [=================>............] - ETA: 205s - loss: 0.2361 - acc: 0.90902071
file 63 : train_test_files/Xy_train64_sel5_doms.npz
2092/3288 [==================>...........] - ETA: 199s - loss: 0.2356 - acc: 0.90932102
file 64 : train_test_files/Xy_train65_sel5_doms.npz
2125/3288 [==================>...........] - ETA: 194s - loss: 0.2349 - acc: 0.90962135
file 65 : train_test_files/Xy_train66_sel5_doms.npz
2158/3288 [=========

 546/3288 [===>..........................] - ETA: 467s - loss: 0.2042 - acc: 0.9234556
file 17 : train_test_files/Xy_train18_sel5_doms.npz
 577/3288 [====>.........................] - ETA: 462s - loss: 0.2036 - acc: 0.9233588
file 18 : train_test_files/Xy_train19_sel5_doms.npz
 611/3288 [====>.........................] - ETA: 455s - loss: 0.2031 - acc: 0.9235621
file 19 : train_test_files/Xy_train20_sel5_doms.npz
 644/3288 [====>.........................] - ETA: 448s - loss: 0.2028 - acc: 0.9235654
file 20 : train_test_files/Xy_train21_sel5_doms.npz
 677/3288 [=====>........................] - ETA: 443s - loss: 0.2028 - acc: 0.9233687
file 21 : train_test_files/Xy_train22_sel5_doms.npz
 710/3288 [=====>........................] - ETA: 437s - loss: 0.2030 - acc: 0.9231720
file 22 : train_test_files/Xy_train23_sel5_doms.npz
 743/3288 [=====>........................] - ETA: 431s - loss: 0.2028 - acc: 0.9231753
file 23 : train_test_files/Xy_train24_sel5_doms.npz
 774/3288 [======>.........

2486/3288 [=====================>........] - ETA: 134s - loss: 0.1993 - acc: 0.92512497
file 76 : train_test_files/Xy_train77_sel5_doms.npz
2520/3288 [=====================>........] - ETA: 128s - loss: 0.1993 - acc: 0.92522531
file 77 : train_test_files/Xy_train78_sel5_doms.npz
2554/3288 [======================>.......] - ETA: 123s - loss: 0.1992 - acc: 0.92512564
file 78 : train_test_files/Xy_train79_sel5_doms.npz
2587/3288 [======================>.......] - ETA: 117s - loss: 0.1995 - acc: 0.92502597
file 79 : train_test_files/Xy_train80_sel5_doms.npz
2620/3288 [======================>.......] - ETA: 112s - loss: 0.1994 - acc: 0.92502630
file 80 : train_test_files/Xy_train81_sel5_doms.npz
2653/3288 [=======================>......] - ETA: 106s - loss: 0.1994 - acc: 0.92502663
file 81 : train_test_files/Xy_train82_sel5_doms.npz
2686/3288 [=======================>......] - ETA: 100s - loss: 0.1995 - acc: 0.92492696
file 82 : train_test_files/Xy_train83_sel5_doms.npz
2719/3288 [=========

1107/3288 [=========>....................] - ETA: 367s - loss: 0.1945 - acc: 0.92671117
file 34 : train_test_files/Xy_train35_sel5_doms.npz
1138/3288 [=========>....................] - ETA: 362s - loss: 0.1943 - acc: 0.92691149
file 35 : train_test_files/Xy_train36_sel5_doms.npz
1170/3288 [=========>....................] - ETA: 357s - loss: 0.1947 - acc: 0.92681181
file 36 : train_test_files/Xy_train37_sel5_doms.npz
1204/3288 [=========>....................] - ETA: 351s - loss: 0.1941 - acc: 0.92711215
file 37 : train_test_files/Xy_train38_sel5_doms.npz
1238/3288 [==========>...................] - ETA: 345s - loss: 0.1942 - acc: 0.92711248
file 38 : train_test_files/Xy_train39_sel5_doms.npz
1271/3288 [==========>...................] - ETA: 339s - loss: 0.1937 - acc: 0.92711281
file 39 : train_test_files/Xy_train40_sel5_doms.npz
1304/3288 [==========>...................] - ETA: 333s - loss: 0.1931 - acc: 0.92761314
file 40 : train_test_files/Xy_train41_sel5_doms.npz
1337/3288 [=========

...

1204/3288 [=========>....................] - ETA: 351s - loss: 0.0552 - acc: 0.97781215
file 37 : train_test_files/Xy_train38_sel5_doms.npz
1238/3288 [==========>...................] - ETA: 345s - loss: 0.0549 - acc: 0.97791248
file 38 : train_test_files/Xy_train39_sel5_doms.npz
1271/3288 [==========>...................] - ETA: 339s - loss: 0.0543 - acc: 0.97811281
file 39 : train_test_files/Xy_train40_sel5_doms.npz
1304/3288 [==========>...................] - ETA: 334s - loss: 0.0539 - acc: 0.97831314
file 40 : train_test_files/Xy_train41_sel5_doms.npz
1337/3288 [===========>..................] - ETA: 329s - loss: 0.0543 - acc: 0.97821348
file 41 : train_test_files/Xy_train42_sel5_doms.npz
1371/3288 [===========>..................] - ETA: 323s - loss: 0.0544 - acc: 0.97821381
file 42 : train_test_files/Xy_train43_sel5_doms.npz
1404/3288 [===========>..................] - ETA: 317s - loss: 0.0544 - acc: 0.97821414
file 43 : train_test_files/Xy_train44_sel5_doms.npz
1435/3288 [=========

3146/3288 [===========================>..] - ETA: 23s - loss: 0.0530 - acc: 0.97913156
file 96 : train_test_files/Xy_train97_sel5_doms.npz
3177/3288 [===========================>..] - ETA: 18s - loss: 0.0530 - acc: 0.97913188
file 97 : train_test_files/Xy_train98_sel5_doms.npz
3211/3288 [============================>.] - ETA: 12s - loss: 0.0528 - acc: 0.97923222
file 98 : train_test_files/Xy_train99_sel5_doms.npz
3245/3288 [============================>.] - ETA: 7s - loss: 0.0527 - acc: 0.97923255
file 99 : train_test_files/Xy_train100_sel5_doms.npz
3278/3288 [============================>.] - ETA: 1s - loss: 0.0527 - acc: 0.97923288
file 0 : train_test_files/Xy_train1_sel5_doms.npz
3288/3288 [==============================] - 549s - loss: 0.0527 - acc: 0.9792   
Epoch 29/30
  21/3288 [..............................] - ETA: 780s - loss: 0.0458 - acc: 0.979232
file 1 : train_test_files/Xy_train2_sel5_doms.npz
  55/3288 [..............................] - ETA: 621s - loss: 0.0599 - acc: 0

1765/3288 [===============>..............] - ETA: 256s - loss: 0.0491 - acc: 0.98041776
file 54 : train_test_files/Xy_train55_sel5_doms.npz
1797/3288 [===============>..............] - ETA: 250s - loss: 0.0493 - acc: 0.98031808
file 55 : train_test_files/Xy_train56_sel5_doms.npz
1829/3288 [===============>..............] - ETA: 245s - loss: 0.0495 - acc: 0.98031840
file 56 : train_test_files/Xy_train57_sel5_doms.npz
1863/3288 [===============>..............] - ETA: 240s - loss: 0.0495 - acc: 0.98031873
file 57 : train_test_files/Xy_train58_sel5_doms.npz
1896/3288 [================>.............] - ETA: 234s - loss: 0.0496 - acc: 0.98031906
file 58 : train_test_files/Xy_train59_sel5_doms.npz
1929/3288 [================>.............] - ETA: 228s - loss: 0.0495 - acc: 0.98031939
file 59 : train_test_files/Xy_train60_sel5_doms.npz
1960/3288 [================>.............] - ETA: 223s - loss: 0.0495 - acc: 0.98021971
file 60 : train_test_files/Xy_train61_sel5_doms.npz
1994/3288 [=========

 384/3288 [==>...........................] - ETA: 489s - loss: 0.0508 - acc: 0.9809394
file 12 : train_test_files/Xy_train13_sel5_doms.npz
 415/3288 [==>...........................] - ETA: 482s - loss: 0.0498 - acc: 0.9809425
file 13 : train_test_files/Xy_train14_sel5_doms.npz
 446/3288 [===>..........................] - ETA: 477s - loss: 0.0496 - acc: 0.9810457
file 14 : train_test_files/Xy_train15_sel5_doms.npz
 480/3288 [===>..........................] - ETA: 471s - loss: 0.0501 - acc: 0.9808490
file 15 : train_test_files/Xy_train16_sel5_doms.npz
 513/3288 [===>..........................] - ETA: 465s - loss: 0.0508 - acc: 0.9806523
file 16 : train_test_files/Xy_train17_sel5_doms.npz
 546/3288 [===>..........................] - ETA: 458s - loss: 0.0507 - acc: 0.9808556
file 17 : train_test_files/Xy_train18_sel5_doms.npz
 577/3288 [====>.........................] - ETA: 454s - loss: 0.0496 - acc: 0.9811588
file 18 : train_test_files/Xy_train19_sel5_doms.npz
 611/3288 [====>...........

2323/3288 [====================>.........] - ETA: 161s - loss: 0.0448 - acc: 0.98252333
file 71 : train_test_files/Xy_train72_sel5_doms.npz
2356/3288 [====================>.........] - ETA: 155s - loss: 0.0450 - acc: 0.98242366
file 72 : train_test_files/Xy_train73_sel5_doms.npz
2389/3288 [====================>.........] - ETA: 150s - loss: 0.0450 - acc: 0.98242399
file 73 : train_test_files/Xy_train74_sel5_doms.npz
2422/3288 [=====================>........] - ETA: 144s - loss: 0.0450 - acc: 0.98242432
file 74 : train_test_files/Xy_train75_sel5_doms.npz
2455/3288 [=====================>........] - ETA: 139s - loss: 0.0451 - acc: 0.98242465
file 75 : train_test_files/Xy_train76_sel5_doms.npz
2486/3288 [=====================>........] - ETA: 133s - loss: 0.0451 - acc: 0.98242497
file 76 : train_test_files/Xy_train77_sel5_doms.npz
2520/3288 [=====================>........] - ETA: 128s - loss: 0.0452 - acc: 0.98242531
file 77 : train_test_files/Xy_train78_sel5_doms.npz
2554/3288 [=========

In [18]:
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_sep_model = vgg_separate_channels(**fargs)

fitargs = {'steps_per_epoch' : 328, 'callbacks' : [early_stop_noval, 
                                                   ModelCheckpoint("models/vgg_sep_ZT_XY_strat_10files.hdf5")],
           'epochs': 30, 'verbose': 1}
history_vgg_sep = vgg_sep_model.fit_generator(Xy_gen_ZT_XY(fnames_train, batch_size=64), **fitargs)


Epoch 1/30
file 0 : train_test_files/Xy_train1_sel5_doms.npz
 21/328 [>.............................] - ETA: 103s - loss: 0.6045 - acc: 0.701632
file 1 : train_test_files/Xy_train2_sel5_doms.npz
 55/328 [====>.........................] - ETA: 62s - loss: 0.4891 - acc: 0.780465
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 49s - loss: 0.4178 - acc: 0.820898
file 3 : train_test_files/Xy_train4_sel5_doms.npz
119/328 [=========>....................] - ETA: 41s - loss: 0.3929 - acc: 0.8336130
file 4 : train_test_files/Xy_train5_sel5_doms.npz
153/328 [============>.................] - ETA: 33s - loss: 0.3718 - acc: 0.8421163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
186/328 [================>.............] - ETA: 26s - loss: 0.3557 - acc: 0.8505196
file 6 : train_test_files/Xy_train7_sel5_doms.npz
217/328 [==================>...........] - ETA: 20s - loss: 0.3417 - acc: 0.8567228
file 7 : train_test_files/Xy_train8_sel5_doms.npz
251

223/328 [===================>..........] - ETA: 18s - loss: 0.2149 - acc: 0.92041873
file 57 : train_test_files/Xy_train58_sel5_doms.npz
256/328 [======================>.......] - ETA: 12s - loss: 0.2134 - acc: 0.92131906
file 58 : train_test_files/Xy_train59_sel5_doms.npz
289/328 [=========================>....] - ETA: 6s - loss: 0.2137 - acc: 0.92111939
file 59 : train_test_files/Xy_train60_sel5_doms.npz
320/328 [============================>.] - ETA: 1s - loss: 0.2139 - acc: 0.92041971
file 60 : train_test_files/Xy_train61_sel5_doms.npz
328/328 [==============================] - 55s - loss: 0.2135 - acc: 0.9206    
Epoch 7/30
 26/328 [=>............................] - ETA: 61s - loss: 0.2111 - acc: 0.92012004
file 61 : train_test_files/Xy_train62_sel5_doms.npz
 59/328 [====>.........................] - ETA: 49s - loss: 0.2229 - acc: 0.91152038
file 62 : train_test_files/Xy_train63_sel5_doms.npz
 93/328 [=======>......................] - ETA: 41s - loss: 0.2172 - acc: 0.91412071
file

 95/328 [=======>......................] - ETA: 39s - loss: 0.2091 - acc: 0.9213425
file 13 : train_test_files/Xy_train14_sel5_doms.npz
126/328 [==========>...................] - ETA: 34s - loss: 0.2085 - acc: 0.9206457
file 14 : train_test_files/Xy_train15_sel5_doms.npz
160/328 [=============>................] - ETA: 28s - loss: 0.2099 - acc: 0.9188490
file 15 : train_test_files/Xy_train16_sel5_doms.npz
193/328 [================>.............] - ETA: 22s - loss: 0.2088 - acc: 0.9198523
file 16 : train_test_files/Xy_train17_sel5_doms.npz
226/328 [===================>..........] - ETA: 17s - loss: 0.2081 - acc: 0.9196556
file 17 : train_test_files/Xy_train18_sel5_doms.npz
257/328 [======================>.......] - ETA: 11s - loss: 0.2070 - acc: 0.9206588
file 18 : train_test_files/Xy_train19_sel5_doms.npz
291/328 [=========================>....] - ETA: 6s - loss: 0.2078 - acc: 0.9202621
file 19 : train_test_files/Xy_train20_sel5_doms.npz
324/328 [============================>.] - ETA: 0

328/328 [==============================] - 54s - loss: 0.2015 - acc: 0.9236    
Epoch 18/30
  1/328 [..............................] - ETA: 12s - loss: 0.2381 - acc: 0.90622300
file 70 : train_test_files/Xy_train71_sel5_doms.npz
 35/328 [==>...........................] - ETA: 47s - loss: 0.1734 - acc: 0.93292333
file 71 : train_test_files/Xy_train72_sel5_doms.npz
 68/328 [=====>........................] - ETA: 42s - loss: 0.1805 - acc: 0.93182366
file 72 : train_test_files/Xy_train73_sel5_doms.npz
101/328 [========>.....................] - ETA: 37s - loss: 0.1823 - acc: 0.93032399
file 73 : train_test_files/Xy_train74_sel5_doms.npz
134/328 [===========>..................] - ETA: 32s - loss: 0.1828 - acc: 0.93022432
file 74 : train_test_files/Xy_train75_sel5_doms.npz
167/328 [==============>...............] - ETA: 26s - loss: 0.1861 - acc: 0.92912465
file 75 : train_test_files/Xy_train76_sel5_doms.npz
198/328 [=================>............] - ETA: 21s - loss: 0.1868 - acc: 0.92992497
f

201/328 [=================>............] - ETA: 20s - loss: 0.2023 - acc: 0.9223851
file 26 : train_test_files/Xy_train27_sel5_doms.npz
234/328 [====================>.........] - ETA: 15s - loss: 0.2019 - acc: 0.9221884
file 27 : train_test_files/Xy_train28_sel5_doms.npz
267/328 [=======================>......] - ETA: 10s - loss: 0.2012 - acc: 0.9227917
file 28 : train_test_files/Xy_train29_sel5_doms.npz
300/328 [==========================>...] - ETA: 4s - loss: 0.2001 - acc: 0.9221950
file 29 : train_test_files/Xy_train30_sel5_doms.npz
328/328 [==============================] - 54s - loss: 0.1990 - acc: 0.9229    
Epoch 24/30
  5/328 [..............................] - ETA: 12s - loss: 0.1657 - acc: 0.9375983
file 30 : train_test_files/Xy_train31_sel5_doms.npz
 38/328 [==>...........................] - ETA: 46s - loss: 0.1852 - acc: 0.93141016
file 31 : train_test_files/Xy_train32_sel5_doms.npz
 71/328 [=====>........................] - ETA: 41s - loss: 0.1886 - acc: 0.92941050
file 32

 78/328 [======>.......................] - ETA: 36s - loss: 0.1969 - acc: 0.92382696
file 82 : train_test_files/Xy_train83_sel5_doms.npz
111/328 [=========>....................] - ETA: 33s - loss: 0.1999 - acc: 0.92182729
file 83 : train_test_files/Xy_train84_sel5_doms.npz
144/328 [============>.................] - ETA: 28s - loss: 0.1937 - acc: 0.92522762
file 84 : train_test_files/Xy_train85_sel5_doms.npz
175/328 [===============>..............] - ETA: 24s - loss: 0.1919 - acc: 0.92572794
file 85 : train_test_files/Xy_train86_sel5_doms.npz
209/328 [==================>...........] - ETA: 18s - loss: 0.1916 - acc: 0.92602827
file 86 : train_test_files/Xy_train87_sel5_doms.npz
240/328 [====================>.........] - ETA: 14s - loss: 0.1912 - acc: 0.92652859
file 87 : train_test_files/Xy_train88_sel5_doms.npz
272/328 [=======================>......] - ETA: 9s - loss: 0.1920 - acc: 0.92612891
file 88 : train_test_files/Xy_train89_sel5_doms.npz
306/328 [==========================>...] -

In [19]:
ytest=[]
for i, (X_test, y_test) in enumerate(Xy_gen_ZT_XY(fnames_test)):
    ytest.append(y_test)
    if i==86:
        break
y_real = np.vstack(np.asarray(ytest))
y_real.shape

file 0 : train_test_files/Xy_test1_sel5_doms.npz
8
file 1 : train_test_files/Xy_test2_sel5_doms.npz
17
file 2 : train_test_files/Xy_test3_sel5_doms.npz
26
file 3 : train_test_files/Xy_test4_sel5_doms.npz
34
file 4 : train_test_files/Xy_test5_sel5_doms.npz
43
file 5 : train_test_files/Xy_test6_sel5_doms.npz
52
file 6 : train_test_files/Xy_test7_sel5_doms.npz
60
file 7 : train_test_files/Xy_test8_sel5_doms.npz
69
file 8 : train_test_files/Xy_test9_sel5_doms.npz
78
file 9 : train_test_files/Xy_test10_sel5_doms.npz


(5179, 2)

In [20]:
pred = vgg_sep_model.predict_generator(Xy_gen_ZT_XY(fnames_test),steps = 87)
ppred = np.vstack(pred)
ppred.shape

file 0 : train_test_files/Xy_test1_sel5_doms.npz
8
file 1 : train_test_files/Xy_test2_sel5_doms.npz
17
file 2 : train_test_files/Xy_test3_sel5_doms.npz
26
file 3 : train_test_files/Xy_test4_sel5_doms.npz
34
file 4 : train_test_files/Xy_test5_sel5_doms.npz
43
file 5 : train_test_files/Xy_test6_sel5_doms.npz
52
file 6 : train_test_files/Xy_test7_sel5_doms.npz
60
file 7 : train_test_files/Xy_test8_sel5_doms.npz
69
file 8 : train_test_files/Xy_test9_sel5_doms.npz
78
file 9 : train_test_files/Xy_test10_sel5_doms.npz
87
file 10 : train_test_files/Xy_test11_sel5_doms.npz


(5179, 2)

In [21]:
predicted_y = np.argmax(pred, axis=1)
from sklearn.metrics import accuracy_score
score = accuracy_score(np.argmax(y_real, axis=1), predicted_y)
score

0.92739911179764434

###  Testing larger kernels

In [67]:
from keras.layers import Input
from keras.layers import concatenate, add
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.core import Flatten, Dense

from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

def vgg_separate_channels(num_classes, optimizer, include_dense=False):
    """VGG inspired Convolutional Networks with multiple inputs
    
    Parameters
    ----------
    num_classes : int
        Number of classes to predict
            
    optimizer : keras.optimizers.Optimizer
        Instance of Keras optimizer to attach to the resulting network
        
    include_dense : bool (default False)
        Flag indicating wether to include a Dense Layer at the top 
        of each separate vgg-like sub-network.
    """
    
    def _conv_block(input_layer, include_dense=False):
        """VGG-inspired Convolutional Blocks"""
        
        prefix = input_layer.name.split('_')[0]

        # Block 1
        x = Conv2D(32, kernel_size=(6, 6), activation='relu', 
                   padding='same', name='{}_block1_conv1'.format(prefix))(input_layer)
        
        x = Conv2D(32, kernel_size=(6, 6), activation='relu', 
                   padding='same', name='{}_block1_conv2'.format(prefix))(x)
        
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), 
                         padding='same', name='{}_block1_pool'.format(prefix))(x)

        # Block 2
        x = Conv2D(64, kernel_size=(6, 6), activation='relu', 
                   padding='same', name='{}_block2_conv1'.format(prefix))(x)
        
        x = Conv2D(64, kernel_size=(6, 6), activation='relu', 
                   padding='same', name='{}_block2_conv2'.format(prefix))(x)
        
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), 
                         padding='same', name='{}_block2_pool'.format(prefix))(x)

        # Block 3
        x = Conv2D(128, kernel_size=(6, 6), activation='relu', 
                   padding='same', name='{}_block3_conv2'.format(prefix))(x)
        
        x = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), 
                             padding='same', name='{}_block3_pool'.format(prefix))(x)
        x = Flatten()(x)
        if include_dense:
            x = Dense(512)(x)
        return x


    ZT_SHAPE = (1, 75, 18)
    XY_SHAPE = (1, 16, 15)
    
    zt_layer = Input(shape=ZT_SHAPE, name='zt_input')
    xy_layer = Input(shape=XY_SHAPE, name='xy_input')
    
    zt_branch = _conv_block(zt_layer, include_dense)
    xy_branch = _conv_block(xy_layer, include_dense)
    
    # top
    if include_dense:
        x = add([zt_branch, xy_branch], name='merge_add')
    else:
        x = concatenate([zt_branch, xy_branch], name='merge_concat')
        
    x = Dense(256,activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[zt_layer, xy_layer], outputs=predictions, name='vgglike')
    model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])
    return model

In [72]:
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_sep_model = vgg_separate_channels(**fargs)

fitargs = {'steps_per_epoch' : 328, 'callbacks' : [early_stop_noval, ModelCheckpoint("vgg_sep_ZT_XY_6x6_10files_100epochs.hdf5")],
           'epochs': 100, 'verbose': 1}
history_vgg_sep = vgg_sep_model.fit_generator(Xy_gen_ZT_XY(fnames_train, batch_size=64), **fitargs)


file 0 : train_test_files/Xy_train1_sel5_doms.npzEpoch 1/100

 22/328 [=>............................] - ETA: 80s - loss: 0.6025 - acc: 0.683232
file 1 : train_test_files/Xy_train2_sel5_doms.npz
 55/328 [====>.........................] - ETA: 56s - loss: 0.4757 - acc: 0.775065
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 45s - loss: 0.4289 - acc: 0.806798
file 3 : train_test_files/Xy_train4_sel5_doms.npz
119/328 [=========>....................] - ETA: 39s - loss: 0.4041 - acc: 0.8233130
file 4 : train_test_files/Xy_train5_sel5_doms.npz
152/328 [============>.................] - ETA: 32s - loss: 0.3757 - acc: 0.8380163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
186/328 [================>.............] - ETA: 25s - loss: 0.3601 - acc: 0.8459196
file 6 : train_test_files/Xy_train7_sel5_doms.npz
217/328 [==================>...........] - ETA: 20s - loss: 0.3452 - acc: 0.8535228
file 7 : train_test_files/Xy_train8_sel5_doms.npz
251

223/328 [===================>..........] - ETA: 18s - loss: 0.2172 - acc: 0.91821873
file 57 : train_test_files/Xy_train58_sel5_doms.npz
256/328 [======================>.......] - ETA: 12s - loss: 0.2168 - acc: 0.91871906
file 58 : train_test_files/Xy_train59_sel5_doms.npz
289/328 [=========================>....] - ETA: 6s - loss: 0.2144 - acc: 0.92001939
file 59 : train_test_files/Xy_train60_sel5_doms.npz
320/328 [============================>.] - ETA: 1s - loss: 0.2135 - acc: 0.92001971
file 60 : train_test_files/Xy_train61_sel5_doms.npz
328/328 [==============================] - 55s - loss: 0.2131 - acc: 0.9201    
Epoch 7/100
 26/328 [=>............................] - ETA: 60s - loss: 0.1865 - acc: 0.92212004
file 61 : train_test_files/Xy_train62_sel5_doms.npz
 59/328 [====>.........................] - ETA: 49s - loss: 0.1992 - acc: 0.92142038
file 62 : train_test_files/Xy_train63_sel5_doms.npz
 92/328 [=======>......................] - ETA: 42s - loss: 0.2085 - acc: 0.91912071
fil

 95/328 [=======>......................] - ETA: 41s - loss: 0.2059 - acc: 0.9233425
file 13 : train_test_files/Xy_train14_sel5_doms.npz
126/328 [==========>...................] - ETA: 35s - loss: 0.2040 - acc: 0.9243457
file 14 : train_test_files/Xy_train15_sel5_doms.npz
160/328 [=============>................] - ETA: 29s - loss: 0.2058 - acc: 0.9227490
file 15 : train_test_files/Xy_train16_sel5_doms.npz
193/328 [================>.............] - ETA: 23s - loss: 0.2063 - acc: 0.9231523
file 16 : train_test_files/Xy_train17_sel5_doms.npz
226/328 [===================>..........] - ETA: 17s - loss: 0.2057 - acc: 0.9226556
file 17 : train_test_files/Xy_train18_sel5_doms.npz
257/328 [======================>.......] - ETA: 12s - loss: 0.2039 - acc: 0.9231588
file 18 : train_test_files/Xy_train19_sel5_doms.npz
291/328 [=========================>....] - ETA: 6s - loss: 0.2024 - acc: 0.9232621
file 19 : train_test_files/Xy_train20_sel5_doms.npz
324/328 [============================>.] - ETA: 0

328/328 [==============================] - 55s - loss: 0.1933 - acc: 0.9278    
Epoch 18/100
  1/328 [..............................] - ETA: 13s - loss: 0.3286 - acc: 0.90622300
file 70 : train_test_files/Xy_train71_sel5_doms.npz
 35/328 [==>...........................] - ETA: 47s - loss: 0.1922 - acc: 0.92862333
file 71 : train_test_files/Xy_train72_sel5_doms.npz
 68/328 [=====>........................] - ETA: 43s - loss: 0.1901 - acc: 0.92582366
file 72 : train_test_files/Xy_train73_sel5_doms.npz
101/328 [========>.....................] - ETA: 38s - loss: 0.1898 - acc: 0.92642399
file 73 : train_test_files/Xy_train74_sel5_doms.npz
134/328 [===========>..................] - ETA: 32s - loss: 0.1902 - acc: 0.92762432
file 74 : train_test_files/Xy_train75_sel5_doms.npz
167/328 [==============>...............] - ETA: 27s - loss: 0.1873 - acc: 0.92862465
file 75 : train_test_files/Xy_train76_sel5_doms.npz
198/328 [=================>............] - ETA: 22s - loss: 0.1864 - acc: 0.92902497


201/328 [=================>............] - ETA: 21s - loss: 0.1952 - acc: 0.9236851
file 26 : train_test_files/Xy_train27_sel5_doms.npz
234/328 [====================>.........] - ETA: 15s - loss: 0.1942 - acc: 0.9248884
file 27 : train_test_files/Xy_train28_sel5_doms.npz
267/328 [=======================>......] - ETA: 10s - loss: 0.1937 - acc: 0.9251917
file 28 : train_test_files/Xy_train29_sel5_doms.npz
300/328 [==========================>...] - ETA: 4s - loss: 0.1932 - acc: 0.9251950
file 29 : train_test_files/Xy_train30_sel5_doms.npz
328/328 [==============================] - 55s - loss: 0.1920 - acc: 0.9263    
Epoch 24/100
  5/328 [..............................] - ETA: 14s - loss: 0.2403 - acc: 0.9219983
file 30 : train_test_files/Xy_train31_sel5_doms.npz
 38/328 [==>...........................] - ETA: 44s - loss: 0.1876 - acc: 0.93371016
file 31 : train_test_files/Xy_train32_sel5_doms.npz
 71/328 [=====>........................] - ETA: 41s - loss: 0.1854 - acc: 0.93381050
file 3

 78/328 [======>.......................] - ETA: 38s - loss: 0.1946 - acc: 0.92942696
file 82 : train_test_files/Xy_train83_sel5_doms.npz
111/328 [=========>....................] - ETA: 33s - loss: 0.1924 - acc: 0.93032729
file 83 : train_test_files/Xy_train84_sel5_doms.npz
144/328 [============>.................] - ETA: 29s - loss: 0.1884 - acc: 0.93052762
file 84 : train_test_files/Xy_train85_sel5_doms.npz
175/328 [===============>..............] - ETA: 24s - loss: 0.1903 - acc: 0.92962794
file 85 : train_test_files/Xy_train86_sel5_doms.npz
209/328 [==================>...........] - ETA: 19s - loss: 0.1872 - acc: 0.93052827
file 86 : train_test_files/Xy_train87_sel5_doms.npz
241/328 [=====================>........] - ETA: 14s - loss: 0.1880 - acc: 0.92982859
file 87 : train_test_files/Xy_train88_sel5_doms.npz
272/328 [=======================>......] - ETA: 9s - loss: 0.1899 - acc: 0.92872891
file 88 : train_test_files/Xy_train89_sel5_doms.npz
306/328 [==========================>...] -

311/328 [===========================>..] - ETA: 2s - loss: 0.1830 - acc: 0.93231281
file 39 : train_test_files/Xy_train40_sel5_doms.npz
328/328 [==============================] - 56s - loss: 0.1812 - acc: 0.9333    
Epoch 35/100
 15/328 [>.............................] - ETA: 13s - loss: 0.1791 - acc: 0.93331314
file 40 : train_test_files/Xy_train41_sel5_doms.npz
 49/328 [===>..........................] - ETA: 37s - loss: 0.1851 - acc: 0.93101348
file 41 : train_test_files/Xy_train42_sel5_doms.npz
 83/328 [======>.......................] - ETA: 35s - loss: 0.1816 - acc: 0.93341381
file 42 : train_test_files/Xy_train43_sel5_doms.npz
116/328 [=========>....................] - ETA: 32s - loss: 0.1856 - acc: 0.92991414
file 43 : train_test_files/Xy_train44_sel5_doms.npz
147/328 [============>.................] - ETA: 28s - loss: 0.1898 - acc: 0.92901446
file 44 : train_test_files/Xy_train45_sel5_doms.npz
181/328 [===============>..............] - ETA: 23s - loss: 0.1877 - acc: 0.92981479
f

185/328 [===============>..............] - ETA: 23s - loss: 0.1816 - acc: 0.93183123
file 95 : train_test_files/Xy_train96_sel5_doms.npz
218/328 [==================>...........] - ETA: 17s - loss: 0.1777 - acc: 0.93413156
file 96 : train_test_files/Xy_train97_sel5_doms.npz
249/328 [=====================>........] - ETA: 13s - loss: 0.1794 - acc: 0.93343188
file 97 : train_test_files/Xy_train98_sel5_doms.npz
283/328 [========================>.....] - ETA: 7s - loss: 0.1786 - acc: 0.93403222
file 98 : train_test_files/Xy_train99_sel5_doms.npz
317/328 [===========================>..] - ETA: 1s - loss: 0.1778 - acc: 0.93423255
file 99 : train_test_files/Xy_train100_sel5_doms.npz
328/328 [==============================] - 57s - loss: 0.1784 - acc: 0.9339    
Epoch 41/100
 21/328 [>.............................] - ETA: 13s - loss: 0.1685 - acc: 0.93753288
file 0 : train_test_files/Xy_train1_sel5_doms.npz
 53/328 [===>..........................] - ETA: 32s - loss: 0.1678 - acc: 0.938732
file 

 60/328 [====>.........................] - ETA: 48s - loss: 0.1854 - acc: 0.92751678
file 51 : train_test_files/Xy_train52_sel5_doms.npz
 93/328 [=======>......................] - ETA: 41s - loss: 0.1857 - acc: 0.92911711
file 52 : train_test_files/Xy_train53_sel5_doms.npz
126/328 [==========>...................] - ETA: 35s - loss: 0.1844 - acc: 0.93021744
file 53 : train_test_files/Xy_train54_sel5_doms.npz
157/328 [=============>................] - ETA: 29s - loss: 0.1824 - acc: 0.93131776
file 54 : train_test_files/Xy_train55_sel5_doms.npz
189/328 [================>.............] - ETA: 24s - loss: 0.1846 - acc: 0.93031808
file 55 : train_test_files/Xy_train56_sel5_doms.npz
221/328 [===================>..........] - ETA: 18s - loss: 0.1866 - acc: 0.92951840
file 56 : train_test_files/Xy_train57_sel5_doms.npz
255/328 [======================>.......] - ETA: 12s - loss: 0.1858 - acc: 0.93021873
file 57 : train_test_files/Xy_train58_sel5_doms.npz
288/328 [=========================>....] 

291/328 [=========================>....] - ETA: 6s - loss: 0.1775 - acc: 0.9332261
file 8 : train_test_files/Xy_train9_sel5_doms.npz
324/328 [============================>.] - ETA: 0s - loss: 0.1781 - acc: 0.9335295
file 9 : train_test_files/Xy_train10_sel5_doms.npz
328/328 [==============================] - 56s - loss: 0.1779 - acc: 0.9337    
Epoch 52/100
 30/328 [=>............................] - ETA: 53s - loss: 0.1765 - acc: 0.9370328
file 10 : train_test_files/Xy_train11_sel5_doms.npz
 62/328 [====>.........................] - ETA: 46s - loss: 0.1830 - acc: 0.9326361
file 11 : train_test_files/Xy_train12_sel5_doms.npz
 96/328 [=======>......................] - ETA: 39s - loss: 0.1805 - acc: 0.9348394
file 12 : train_test_files/Xy_train13_sel5_doms.npz
127/328 [==========>...................] - ETA: 34s - loss: 0.1823 - acc: 0.9336425
file 13 : train_test_files/Xy_train14_sel5_doms.npz
158/328 [=============>................] - ETA: 29s - loss: 0.1825 - acc: 0.9334457
file 14 : tr

In [73]:
ytest=[]
for i, (X_test, y_test) in enumerate(Xy_gen_ZT_XY(fnames_test)):
    ytest.append(y_test)
    if i==87:
        break
y_real = np.vstack(np.asarray(ytest))
y_real.shape

pred = vgg_sep_model.predict_generator(Xy_gen_ZT_XY(fnames_test),steps = 88)
ppred = np.vstack(pred)
ppred.shape


predicted_y = np.argmax(pred, axis=1)
from sklearn.metrics import accuracy_score
score = accuracy_score(np.argmax(y_real, axis=1), predicted_y)
score

file 0 : train_test_files/Xy_test1_sel5_doms.npz
8
file 1 : train_test_files/Xy_test2_sel5_doms.npz
17
file 2 : train_test_files/Xy_test3_sel5_doms.npz
26
file 3 : train_test_files/Xy_test4_sel5_doms.npz
34
file 4 : train_test_files/Xy_test5_sel5_doms.npz
43
file 5 : train_test_files/Xy_test6_sel5_doms.npz
52
file 6 : train_test_files/Xy_test7_sel5_doms.npz
60
file 7 : train_test_files/Xy_test8_sel5_doms.npz
69
file 8 : train_test_files/Xy_test9_sel5_doms.npz
78
file 9 : train_test_files/Xy_test10_sel5_doms.npz
87
file 10 : train_test_files/Xy_test11_sel5_doms.npz
file 0 : train_test_files/Xy_test1_sel5_doms.npz
8
file 1 : train_test_files/Xy_test2_sel5_doms.npz
17
file 2 : train_test_files/Xy_test3_sel5_doms.npz
26
file 3 : train_test_files/Xy_test4_sel5_doms.npz
34
file 4 : train_test_files/Xy_test5_sel5_doms.npz
43
file 5 : train_test_files/Xy_test6_sel5_doms.npz
52
file 6 : train_test_files/Xy_test7_sel5_doms.npz
60
file 7 : train_test_files/Xy_test8_sel5_doms.npz
69
file 8 : train

0.9282853328247187

In [74]:
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_sep_model = vgg_separate_channels(**fargs)

fitargs = {'steps_per_epoch' : 3288, 'callbacks' : [early_stop_noval, ModelCheckpoint("vgg_sep_ZT_XY_6x6_100files_50epochs.hdf5")],
           'epochs': 50, 'verbose': 1}
history_vgg_sep = vgg_sep_model.fit_generator(Xy_gen_ZT_XY(fnames_train, batch_size=64), **fitargs)


Epoch 1/50file 0 : train_test_files/Xy_train1_sel5_doms.npz

  21/3288 [..............................] - ETA: 894s - loss: 0.6514 - acc: 0.682332
file 1 : train_test_files/Xy_train2_sel5_doms.npz
  55/3288 [..............................] - ETA: 668s - loss: 0.4985 - acc: 0.771065
file 2 : train_test_files/Xy_train3_sel5_doms.npz
  88/3288 [..............................] - ETA: 627s - loss: 0.4424 - acc: 0.806198
file 3 : train_test_files/Xy_train4_sel5_doms.npz
 119/3288 [>.............................] - ETA: 602s - loss: 0.4122 - acc: 0.8251130
file 4 : train_test_files/Xy_train5_sel5_doms.npz
 153/3288 [>.............................] - ETA: 580s - loss: 0.3807 - acc: 0.8398163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
 186/3288 [>.............................] - ETA: 567s - loss: 0.3642 - acc: 0.8472196
file 6 : train_test_files/Xy_train7_sel5_doms.npz
 217/3288 [>.............................] - ETA: 558s - loss: 0.3477 - acc: 0.8552228
file 7 : train_test_files/Xy_trai

1929/3288 [================>.............] - ETA: 233s - loss: 0.2352 - acc: 0.90941939
file 59 : train_test_files/Xy_train60_sel5_doms.npz
1960/3288 [================>.............] - ETA: 228s - loss: 0.2347 - acc: 0.90961971
file 60 : train_test_files/Xy_train61_sel5_doms.npz
1993/3288 [=================>............] - ETA: 222s - loss: 0.2339 - acc: 0.90982004
file 61 : train_test_files/Xy_train62_sel5_doms.npz
2027/3288 [=================>............] - ETA: 216s - loss: 0.2334 - acc: 0.91002038
file 62 : train_test_files/Xy_train63_sel5_doms.npz
2061/3288 [=================>............] - ETA: 210s - loss: 0.2333 - acc: 0.91012071
file 63 : train_test_files/Xy_train64_sel5_doms.npz
2092/3288 [==================>...........] - ETA: 205s - loss: 0.2328 - acc: 0.91032102
file 64 : train_test_files/Xy_train65_sel5_doms.npz
2125/3288 [==================>...........] - ETA: 199s - loss: 0.2321 - acc: 0.91062135
file 65 : train_test_files/Xy_train66_sel5_doms.npz
2158/3288 [=========

 546/3288 [===>..........................] - ETA: 472s - loss: 0.2017 - acc: 0.9245556
file 17 : train_test_files/Xy_train18_sel5_doms.npz
 577/3288 [====>.........................] - ETA: 468s - loss: 0.2010 - acc: 0.9247588
file 18 : train_test_files/Xy_train19_sel5_doms.npz
 611/3288 [====>.........................] - ETA: 461s - loss: 0.2006 - acc: 0.9248621
file 19 : train_test_files/Xy_train20_sel5_doms.npz
 644/3288 [====>.........................] - ETA: 454s - loss: 0.2004 - acc: 0.9249654
file 20 : train_test_files/Xy_train21_sel5_doms.npz
 677/3288 [=====>........................] - ETA: 448s - loss: 0.2004 - acc: 0.9247687
file 21 : train_test_files/Xy_train22_sel5_doms.npz
 710/3288 [=====>........................] - ETA: 442s - loss: 0.2007 - acc: 0.9244720
file 22 : train_test_files/Xy_train23_sel5_doms.npz
 743/3288 [=====>........................] - ETA: 436s - loss: 0.2004 - acc: 0.9244753
file 23 : train_test_files/Xy_train24_sel5_doms.npz
 775/3288 [======>.........

2486/3288 [=====================>........] - ETA: 136s - loss: 0.1965 - acc: 0.92592497
file 76 : train_test_files/Xy_train77_sel5_doms.npz
2520/3288 [=====================>........] - ETA: 130s - loss: 0.1965 - acc: 0.92592531
file 77 : train_test_files/Xy_train78_sel5_doms.npz
2554/3288 [======================>.......] - ETA: 124s - loss: 0.1964 - acc: 0.92582564
file 78 : train_test_files/Xy_train79_sel5_doms.npz
2587/3288 [======================>.......] - ETA: 119s - loss: 0.1966 - acc: 0.92582597
file 79 : train_test_files/Xy_train80_sel5_doms.npz
2620/3288 [======================>.......] - ETA: 113s - loss: 0.1965 - acc: 0.92582630
file 80 : train_test_files/Xy_train81_sel5_doms.npz
2653/3288 [=======================>......] - ETA: 107s - loss: 0.1965 - acc: 0.92582663
file 81 : train_test_files/Xy_train82_sel5_doms.npz
2685/3288 [=======================>......] - ETA: 102s - loss: 0.1966 - acc: 0.92582696
file 82 : train_test_files/Xy_train83_sel5_doms.npz
2719/3288 [=========

1107/3288 [=========>....................] - ETA: 374s - loss: 0.1920 - acc: 0.92751117
file 34 : train_test_files/Xy_train35_sel5_doms.npz
1138/3288 [=========>....................] - ETA: 370s - loss: 0.1916 - acc: 0.92771149
file 35 : train_test_files/Xy_train36_sel5_doms.npz
1170/3288 [=========>....................] - ETA: 364s - loss: 0.1921 - acc: 0.92761181
file 36 : train_test_files/Xy_train37_sel5_doms.npz
1204/3288 [=========>....................] - ETA: 358s - loss: 0.1915 - acc: 0.92791215
file 37 : train_test_files/Xy_train38_sel5_doms.npz
1238/3288 [==========>...................] - ETA: 352s - loss: 0.1915 - acc: 0.92801248
file 38 : train_test_files/Xy_train39_sel5_doms.npz
1271/3288 [==========>...................] - ETA: 346s - loss: 0.1910 - acc: 0.92811281
file 39 : train_test_files/Xy_train40_sel5_doms.npz
1303/3288 [==========>...................] - ETA: 341s - loss: 0.1904 - acc: 0.92851314
file 40 : train_test_files/Xy_train41_sel5_doms.npz
1337/3288 [=========

...

 644/3288 [====>.........................] - ETA: 478s - loss: 0.0134 - acc: 0.9959654
file 20 : train_test_files/Xy_train21_sel5_doms.npz
 676/3288 [=====>........................] - ETA: 473s - loss: 0.0136 - acc: 0.9958687
file 21 : train_test_files/Xy_train22_sel5_doms.npz
 710/3288 [=====>........................] - ETA: 466s - loss: 0.0136 - acc: 0.9959720
file 22 : train_test_files/Xy_train23_sel5_doms.npz
 743/3288 [=====>........................] - ETA: 460s - loss: 0.0136 - acc: 0.9959753
file 23 : train_test_files/Xy_train24_sel5_doms.npz
 774/3288 [======>.......................] - ETA: 455s - loss: 0.0136 - acc: 0.9959785
file 24 : train_test_files/Xy_train25_sel5_doms.npz
 808/3288 [======>.......................] - ETA: 448s - loss: 0.0136 - acc: 0.9959818
file 25 : train_test_files/Xy_train26_sel5_doms.npz
 841/3288 [======>.......................] - ETA: 442s - loss: 0.0135 - acc: 0.9959851
file 26 : train_test_files/Xy_train27_sel5_doms.npz
 874/3288 [======>.........

2587/3288 [======================>.......] - ETA: 125s - loss: 0.0133 - acc: 0.99582597
file 79 : train_test_files/Xy_train80_sel5_doms.npz
2620/3288 [======================>.......] - ETA: 119s - loss: 0.0133 - acc: 0.99582630
file 80 : train_test_files/Xy_train81_sel5_doms.npz
2653/3288 [=======================>......] - ETA: 113s - loss: 0.0132 - acc: 0.99582663
file 81 : train_test_files/Xy_train82_sel5_doms.npz
2686/3288 [=======================>......] - ETA: 107s - loss: 0.0132 - acc: 0.99582696
file 82 : train_test_files/Xy_train83_sel5_doms.npz
2719/3288 [=======================>......] - ETA: 101s - loss: 0.0132 - acc: 0.99582729
file 83 : train_test_files/Xy_train84_sel5_doms.npz
2752/3288 [========================>.....] - ETA: 95s - loss: 0.0132 - acc: 0.99582762
file 84 : train_test_files/Xy_train85_sel5_doms.npz
2783/3288 [========================>.....] - ETA: 90s - loss: 0.0131 - acc: 0.99582794
file 85 : train_test_files/Xy_train86_sel5_doms.npz
2817/3288 [===========

1204/3288 [=========>....................] - ETA: 375s - loss: 0.0129 - acc: 0.99601215
file 37 : train_test_files/Xy_train38_sel5_doms.npz
1238/3288 [==========>...................] - ETA: 369s - loss: 0.0128 - acc: 0.99601248
file 38 : train_test_files/Xy_train39_sel5_doms.npz
1271/3288 [==========>...................] - ETA: 363s - loss: 0.0129 - acc: 0.99601281
file 39 : train_test_files/Xy_train40_sel5_doms.npz
1304/3288 [==========>...................] - ETA: 357s - loss: 0.0130 - acc: 0.99601314
file 40 : train_test_files/Xy_train41_sel5_doms.npz
1337/3288 [===========>..................] - ETA: 351s - loss: 0.0128 - acc: 0.99601348
file 41 : train_test_files/Xy_train42_sel5_doms.npz
1371/3288 [===========>..................] - ETA: 344s - loss: 0.0130 - acc: 0.99591381
file 42 : train_test_files/Xy_train43_sel5_doms.npz
1404/3288 [===========>..................] - ETA: 338s - loss: 0.0130 - acc: 0.99591414
file 43 : train_test_files/Xy_train44_sel5_doms.npz
1435/3288 [=========

3146/3288 [===========================>..] - ETA: 25s - loss: 0.0124 - acc: 0.99603156
file 96 : train_test_files/Xy_train97_sel5_doms.npz
3178/3288 [===========================>..] - ETA: 19s - loss: 0.0123 - acc: 0.99603188
file 97 : train_test_files/Xy_train98_sel5_doms.npz
3211/3288 [============================>.] - ETA: 13s - loss: 0.0123 - acc: 0.99603222
file 98 : train_test_files/Xy_train99_sel5_doms.npz
3245/3288 [============================>.] - ETA: 7s - loss: 0.0123 - acc: 0.99603255
file 99 : train_test_files/Xy_train100_sel5_doms.npz
3278/3288 [============================>.] - ETA: 1s - loss: 0.0123 - acc: 0.99603288
file 0 : train_test_files/Xy_train1_sel5_doms.npz
3288/3288 [==============================] - 586s - loss: 0.0122 - acc: 0.9960   
Epoch 50/50
  21/3288 [..............................] - ETA: 834s - loss: 0.0098 - acc: 0.997032
file 1 : train_test_files/Xy_train2_sel5_doms.npz
  55/3288 [..............................] - ETA: 664s - loss: 0.0103 - acc: 0

1766/3288 [===============>..............] - ETA: 272s - loss: 0.0123 - acc: 0.99581776
file 54 : train_test_files/Xy_train55_sel5_doms.npz
1797/3288 [===============>..............] - ETA: 266s - loss: 0.0123 - acc: 0.99581808
file 55 : train_test_files/Xy_train56_sel5_doms.npz
1829/3288 [===============>..............] - ETA: 260s - loss: 0.0124 - acc: 0.99581840
file 56 : train_test_files/Xy_train57_sel5_doms.npz
1863/3288 [===============>..............] - ETA: 254s - loss: 0.0123 - acc: 0.99581873
file 57 : train_test_files/Xy_train58_sel5_doms.npz
1896/3288 [================>.............] - ETA: 248s - loss: 0.0125 - acc: 0.99581906
file 58 : train_test_files/Xy_train59_sel5_doms.npz
1929/3288 [================>.............] - ETA: 242s - loss: 0.0126 - acc: 0.99581939
file 59 : train_test_files/Xy_train60_sel5_doms.npz
1960/3288 [================>.............] - ETA: 237s - loss: 0.0125 - acc: 0.99581971
file 60 : train_test_files/Xy_train61_sel5_doms.npz
1994/3288 [=========

In [75]:
ytest=[]
for i, (X_test, y_test) in enumerate(Xy_gen_ZT_XY(fnames_test)):
    ytest.append(y_test)
    if i==878:
        break
y_real = np.vstack(np.asarray(ytest))
y_real.shape

pred = vgg_sep_model.predict_generator(Xy_gen_ZT_XY(fnames_test),steps = 879)
ppred = np.vstack(pred)
ppred.shape


predicted_y = np.argmax(pred, axis=1)
from sklearn.metrics import accuracy_score
score = accuracy_score(np.argmax(y_real, axis=1), predicted_y)
score

file 0 : train_test_files/Xy_test1_sel5_doms.npz
8
file 1 : train_test_files/Xy_test2_sel5_doms.npz
17
file 2 : train_test_files/Xy_test3_sel5_doms.npz
26
file 3 : train_test_files/Xy_test4_sel5_doms.npz
34
file 4 : train_test_files/Xy_test5_sel5_doms.npz
43
file 5 : train_test_files/Xy_test6_sel5_doms.npz
52
file 6 : train_test_files/Xy_test7_sel5_doms.npz
60
file 7 : train_test_files/Xy_test8_sel5_doms.npz
69
file 8 : train_test_files/Xy_test9_sel5_doms.npz
78
file 9 : train_test_files/Xy_test10_sel5_doms.npz
87
file 10 : train_test_files/Xy_test11_sel5_doms.npz
96
file 11 : train_test_files/Xy_test12_sel5_doms.npz
105
file 12 : train_test_files/Xy_test13_sel5_doms.npz
113
file 13 : train_test_files/Xy_test14_sel5_doms.npz
121
file 14 : train_test_files/Xy_test15_sel5_doms.npz
130
file 15 : train_test_files/Xy_test16_sel5_doms.npz
139
file 16 : train_test_files/Xy_test17_sel5_doms.npz
148
file 17 : train_test_files/Xy_test18_sel5_doms.npz
156
file 18 : train_test_files/Xy_test19_sel5

449
file 51 : train_test_files/Xy_test52_sel5_doms.npz
458
file 52 : train_test_files/Xy_test53_sel5_doms.npz
467
file 53 : train_test_files/Xy_test54_sel5_doms.npz
475
file 54 : train_test_files/Xy_test55_sel5_doms.npz
483
file 55 : train_test_files/Xy_test56_sel5_doms.npz
491
file 56 : train_test_files/Xy_test57_sel5_doms.npz
500
file 57 : train_test_files/Xy_test58_sel5_doms.npz
509
file 58 : train_test_files/Xy_test59_sel5_doms.npz
518
file 59 : train_test_files/Xy_test60_sel5_doms.npz
526
file 60 : train_test_files/Xy_test61_sel5_doms.npz
535
file 61 : train_test_files/Xy_test62_sel5_doms.npz
544
file 62 : train_test_files/Xy_test63_sel5_doms.npz
553
file 63 : train_test_files/Xy_test64_sel5_doms.npz
561
file 64 : train_test_files/Xy_test65_sel5_doms.npz
570
file 65 : train_test_files/Xy_test66_sel5_doms.npz
579
file 66 : train_test_files/Xy_test67_sel5_doms.npz
588
file 67 : train_test_files/Xy_test68_sel5_doms.npz
597
file 68 : train_test_files/Xy_test69_sel5_doms.npz
606
file 6

0.91192249797367708

### 8x8 kernels

In [78]:

from keras.layers import Input
from keras.layers import concatenate, add
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.core import Flatten, Dense

from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

def vgg_separate_channels(num_classes, optimizer, include_dense=False):
    """VGG inspired Convolutional Networks with multiple inputs
    
    Parameters
    ----------
    num_classes : int
        Number of classes to predict
            
    optimizer : keras.optimizers.Optimizer
        Instance of Keras optimizer to attach to the resulting network
        
    include_dense : bool (default False)
        Flag indicating wether to include a Dense Layer at the top 
        of each separate vgg-like sub-network.
    """
    
    def _conv_block(input_layer, include_dense=False):
        """VGG-inspired Convolutional Blocks"""
        
        prefix = input_layer.name.split('_')[0]

        # Block 1
        x = Conv2D(32, kernel_size=(8, 8), activation='relu', 
                   padding='same', name='{}_block1_conv1'.format(prefix))(input_layer)
        
        x = Conv2D(32, kernel_size=(8, 8), activation='relu', 
                   padding='same', name='{}_block1_conv2'.format(prefix))(x)
        
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), 
                         padding='same', name='{}_block1_pool'.format(prefix))(x)

        # Block 2
        x = Conv2D(64, kernel_size=(6, 6), activation='relu', 
                   padding='same', name='{}_block2_conv1'.format(prefix))(x)
        
        x = Conv2D(64, kernel_size=(6, 6), activation='relu', 
                   padding='same', name='{}_block2_conv2'.format(prefix))(x)
        
        x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), 
                         padding='same', name='{}_block2_pool'.format(prefix))(x)

        # Block 3
        x = Conv2D(128, kernel_size=(3, 3), activation='relu', 
                   padding='same', name='{}_block3_conv2'.format(prefix))(x)
        
        x = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), 
                             padding='same', name='{}_block3_pool'.format(prefix))(x)
        x = Flatten()(x)
        if include_dense:
            x = Dense(512)(x)
        return x


    ZT_SHAPE = (1, 75, 18)
    XY_SHAPE = (1, 16, 15)
    
    zt_layer = Input(shape=ZT_SHAPE, name='zt_input')
    xy_layer = Input(shape=XY_SHAPE, name='xy_input')
    
    zt_branch = _conv_block(zt_layer, include_dense)
    xy_branch = _conv_block(xy_layer, include_dense)
    
    # top
    if include_dense:
        x = add([zt_branch, xy_branch], name='merge_add')
    else:
        x = concatenate([zt_branch, xy_branch], name='merge_concat')
        
    x = Dense(256,activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[zt_layer, xy_layer], outputs=predictions, name='vgglike')
    model.compile(loss=categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])
    return model

In [79]:
fargs = {'num_classes': 2, 'optimizer': Adadelta()}
vgg_sep_model = vgg_separate_channels(**fargs)

fitargs = {'steps_per_epoch' : 328, 'callbacks' : [early_stop_noval, ModelCheckpoint("vgg_sep_ZT_XY_863_10files_50epochs.hdf5")],
           'epochs': 50, 'verbose': 1}
history_vgg_sep = vgg_sep_model.fit_generator(Xy_gen_ZT_XY(fnames_train, batch_size=64), **fitargs)


Epoch 1/50file 0 : train_test_files/Xy_train1_sel5_doms.npz

 21/328 [>.............................] - ETA: 85s - loss: 0.6023 - acc: 0.679332
file 1 : train_test_files/Xy_train2_sel5_doms.npz
 55/328 [====>.........................] - ETA: 57s - loss: 0.4744 - acc: 0.775565
file 2 : train_test_files/Xy_train3_sel5_doms.npz
 88/328 [=======>......................] - ETA: 47s - loss: 0.4271 - acc: 0.808198
file 3 : train_test_files/Xy_train4_sel5_doms.npz
119/328 [=========>....................] - ETA: 39s - loss: 0.4054 - acc: 0.8243130
file 4 : train_test_files/Xy_train5_sel5_doms.npz
153/328 [============>.................] - ETA: 32s - loss: 0.3770 - acc: 0.8388163
file 5 : train_test_files/Xy_train6_sel5_doms.npz
186/328 [================>.............] - ETA: 25s - loss: 0.3635 - acc: 0.8467196
file 6 : train_test_files/Xy_train7_sel5_doms.npz
217/328 [==================>...........] - ETA: 20s - loss: 0.3481 - acc: 0.8542228
file 7 : train_test_files/Xy_train8_sel5_doms.npz
251/

223/328 [===================>..........] - ETA: 18s - loss: 0.2168 - acc: 0.91701873
file 57 : train_test_files/Xy_train58_sel5_doms.npz
256/328 [======================>.......] - ETA: 12s - loss: 0.2160 - acc: 0.91791906
file 58 : train_test_files/Xy_train59_sel5_doms.npz
289/328 [=========================>....] - ETA: 6s - loss: 0.2138 - acc: 0.91891939
file 59 : train_test_files/Xy_train60_sel5_doms.npz
320/328 [============================>.] - ETA: 1s - loss: 0.2135 - acc: 0.91851971
file 60 : train_test_files/Xy_train61_sel5_doms.npz
328/328 [==============================] - 55s - loss: 0.2132 - acc: 0.9186    
Epoch 7/50
 26/328 [=>............................] - ETA: 63s - loss: 0.1846 - acc: 0.92212004
file 61 : train_test_files/Xy_train62_sel5_doms.npz
 59/328 [====>.........................] - ETA: 51s - loss: 0.1976 - acc: 0.92172038
file 62 : train_test_files/Xy_train63_sel5_doms.npz
 93/328 [=======>......................] - ETA: 42s - loss: 0.2076 - acc: 0.92022071
file

 95/328 [=======>......................] - ETA: 40s - loss: 0.2054 - acc: 0.9231425
file 13 : train_test_files/Xy_train14_sel5_doms.npz
126/328 [==========>...................] - ETA: 35s - loss: 0.2033 - acc: 0.9248457
file 14 : train_test_files/Xy_train15_sel5_doms.npz
160/328 [=============>................] - ETA: 28s - loss: 0.2059 - acc: 0.9235490
file 15 : train_test_files/Xy_train16_sel5_doms.npz
192/328 [================>.............] - ETA: 23s - loss: 0.2071 - acc: 0.9234523
file 16 : train_test_files/Xy_train17_sel5_doms.npz
226/328 [===================>..........] - ETA: 17s - loss: 0.2062 - acc: 0.9228556
file 17 : train_test_files/Xy_train18_sel5_doms.npz
257/328 [======================>.......] - ETA: 12s - loss: 0.2042 - acc: 0.9231588
file 18 : train_test_files/Xy_train19_sel5_doms.npz
291/328 [=========================>....] - ETA: 6s - loss: 0.2025 - acc: 0.9235621
file 19 : train_test_files/Xy_train20_sel5_doms.npz
324/328 [============================>.] - ETA: 0

328/328 [==============================] - 55s - loss: 0.1929 - acc: 0.9269    
Epoch 18/50
  1/328 [..............................] - ETA: 13s - loss: 0.3241 - acc: 0.89062300
file 70 : train_test_files/Xy_train71_sel5_doms.npz
 35/328 [==>...........................] - ETA: 50s - loss: 0.1947 - acc: 0.92692333
file 71 : train_test_files/Xy_train72_sel5_doms.npz
 68/328 [=====>........................] - ETA: 43s - loss: 0.1902 - acc: 0.92512366
file 72 : train_test_files/Xy_train73_sel5_doms.npz
101/328 [========>.....................] - ETA: 38s - loss: 0.1907 - acc: 0.92622399
file 73 : train_test_files/Xy_train74_sel5_doms.npz
134/328 [===========>..................] - ETA: 32s - loss: 0.1905 - acc: 0.92762432
file 74 : train_test_files/Xy_train75_sel5_doms.npz
167/328 [==============>...............] - ETA: 27s - loss: 0.1878 - acc: 0.92922465
file 75 : train_test_files/Xy_train76_sel5_doms.npz
198/328 [=================>............] - ETA: 22s - loss: 0.1868 - acc: 0.92952497
f

201/328 [=================>............] - ETA: 21s - loss: 0.1960 - acc: 0.9237851
file 26 : train_test_files/Xy_train27_sel5_doms.npz
234/328 [====================>.........] - ETA: 15s - loss: 0.1949 - acc: 0.9251884
file 27 : train_test_files/Xy_train28_sel5_doms.npz
267/328 [=======================>......] - ETA: 10s - loss: 0.1945 - acc: 0.9259917
file 28 : train_test_files/Xy_train29_sel5_doms.npz
300/328 [==========================>...] - ETA: 4s - loss: 0.1935 - acc: 0.9259950
file 29 : train_test_files/Xy_train30_sel5_doms.npz
328/328 [==============================] - 55s - loss: 0.1920 - acc: 0.9273    
Epoch 24/50
  5/328 [..............................] - ETA: 14s - loss: 0.2421 - acc: 0.9187983
file 30 : train_test_files/Xy_train31_sel5_doms.npz
 38/328 [==>...........................] - ETA: 43s - loss: 0.1877 - acc: 0.93331016
file 31 : train_test_files/Xy_train32_sel5_doms.npz
 71/328 [=====>........................] - ETA: 41s - loss: 0.1845 - acc: 0.93391050
file 32

 78/328 [======>.......................] - ETA: 37s - loss: 0.1931 - acc: 0.92922696
file 82 : train_test_files/Xy_train83_sel5_doms.npz
110/328 [=========>....................] - ETA: 34s - loss: 0.1919 - acc: 0.93062729
file 83 : train_test_files/Xy_train84_sel5_doms.npz
144/328 [============>.................] - ETA: 29s - loss: 0.1871 - acc: 0.93142762
file 84 : train_test_files/Xy_train85_sel5_doms.npz
175/328 [===============>..............] - ETA: 24s - loss: 0.1890 - acc: 0.93072794
file 85 : train_test_files/Xy_train86_sel5_doms.npz
209/328 [==================>...........] - ETA: 19s - loss: 0.1863 - acc: 0.93142827
file 86 : train_test_files/Xy_train87_sel5_doms.npz
240/328 [====================>.........] - ETA: 14s - loss: 0.1866 - acc: 0.93142859
file 87 : train_test_files/Xy_train88_sel5_doms.npz
272/328 [=======================>......] - ETA: 9s - loss: 0.1886 - acc: 0.93032891
file 88 : train_test_files/Xy_train89_sel5_doms.npz
306/328 [==========================>...] -

311/328 [===========================>..] - ETA: 2s - loss: 0.1824 - acc: 0.93291281
file 39 : train_test_files/Xy_train40_sel5_doms.npz
328/328 [==============================] - 55s - loss: 0.1805 - acc: 0.9339    
Epoch 35/50
 15/328 [>.............................] - ETA: 13s - loss: 0.1808 - acc: 0.93851314
file 40 : train_test_files/Xy_train41_sel5_doms.npz
 49/328 [===>..........................] - ETA: 36s - loss: 0.1843 - acc: 0.93171348
file 41 : train_test_files/Xy_train42_sel5_doms.npz
 83/328 [======>.......................] - ETA: 35s - loss: 0.1798 - acc: 0.93391381
file 42 : train_test_files/Xy_train43_sel5_doms.npz
116/328 [=========>....................] - ETA: 32s - loss: 0.1836 - acc: 0.93051414
file 43 : train_test_files/Xy_train44_sel5_doms.npz
147/328 [============>.................] - ETA: 28s - loss: 0.1885 - acc: 0.92941446
file 44 : train_test_files/Xy_train45_sel5_doms.npz
181/328 [===============>..............] - ETA: 23s - loss: 0.1861 - acc: 0.93091479
fi

185/328 [===============>..............] - ETA: 22s - loss: 0.1788 - acc: 0.93353123
file 95 : train_test_files/Xy_train96_sel5_doms.npz
218/328 [==================>...........] - ETA: 17s - loss: 0.1758 - acc: 0.93503156
file 96 : train_test_files/Xy_train97_sel5_doms.npz
250/328 [=====================>........] - ETA: 12s - loss: 0.1773 - acc: 0.93443188
file 97 : train_test_files/Xy_train98_sel5_doms.npz
283/328 [========================>.....] - ETA: 7s - loss: 0.1768 - acc: 0.93503222
file 98 : train_test_files/Xy_train99_sel5_doms.npz
317/328 [===========================>..] - ETA: 1s - loss: 0.1759 - acc: 0.93503255
file 99 : train_test_files/Xy_train100_sel5_doms.npz
328/328 [==============================] - 55s - loss: 0.1767 - acc: 0.9346    
Epoch 41/50
 21/328 [>.............................] - ETA: 13s - loss: 0.1676 - acc: 0.93603288
file 0 : train_test_files/Xy_train1_sel5_doms.npz
 53/328 [===>..........................] - ETA: 33s - loss: 0.1661 - acc: 0.941032
file 1

 60/328 [====>.........................] - ETA: 48s - loss: 0.1854 - acc: 0.93011678
file 51 : train_test_files/Xy_train52_sel5_doms.npz
 93/328 [=======>......................] - ETA: 40s - loss: 0.1832 - acc: 0.93161711
file 52 : train_test_files/Xy_train53_sel5_doms.npz
126/328 [==========>...................] - ETA: 34s - loss: 0.1823 - acc: 0.93241744
file 53 : train_test_files/Xy_train54_sel5_doms.npz
157/328 [=============>................] - ETA: 29s - loss: 0.1795 - acc: 0.93371776
file 54 : train_test_files/Xy_train55_sel5_doms.npz
189/328 [================>.............] - ETA: 24s - loss: 0.1822 - acc: 0.93291808
file 55 : train_test_files/Xy_train56_sel5_doms.npz
221/328 [===================>..........] - ETA: 18s - loss: 0.1846 - acc: 0.93171840
file 56 : train_test_files/Xy_train57_sel5_doms.npz
255/328 [======================>.......] - ETA: 12s - loss: 0.1839 - acc: 0.93171873
file 57 : train_test_files/Xy_train58_sel5_doms.npz
288/328 [=========================>....] 

In [80]:
ytest=[]
for i, (X_test, y_test) in enumerate(Xy_gen_ZT_XY(fnames_test)):
    ytest.append(y_test)
    if i==86:
        break
y_real = np.vstack(np.asarray(ytest))
y_real.shape

pred = vgg_sep_model.predict_generator(Xy_gen_ZT_XY(fnames_test),steps = 87)
ppred = np.vstack(pred)
ppred.shape


predicted_y = np.argmax(pred, axis=1)
from sklearn.metrics import accuracy_score
score = accuracy_score(np.argmax(y_real, axis=1), predicted_y)
score

file 0 : train_test_files/Xy_test1_sel5_doms.npz
8
file 1 : train_test_files/Xy_test2_sel5_doms.npz
17
file 2 : train_test_files/Xy_test3_sel5_doms.npz
26
file 3 : train_test_files/Xy_test4_sel5_doms.npz
34
file 4 : train_test_files/Xy_test5_sel5_doms.npz
43
file 5 : train_test_files/Xy_test6_sel5_doms.npz
52
file 6 : train_test_files/Xy_test7_sel5_doms.npz
60
file 7 : train_test_files/Xy_test8_sel5_doms.npz
69
file 8 : train_test_files/Xy_test9_sel5_doms.npz
78
file 9 : train_test_files/Xy_test10_sel5_doms.npz
file 0 : train_test_files/Xy_test1_sel5_doms.npz
8
file 1 : train_test_files/Xy_test2_sel5_doms.npz
17
file 2 : train_test_files/Xy_test3_sel5_doms.npz
26
file 3 : train_test_files/Xy_test4_sel5_doms.npz
34
file 4 : train_test_files/Xy_test5_sel5_doms.npz
43
file 5 : train_test_files/Xy_test6_sel5_doms.npz
52
file 6 : train_test_files/Xy_test7_sel5_doms.npz
60
file 7 : train_test_files/Xy_test8_sel5_doms.npz
69
file 8 : train_test_files/Xy_test9_sel5_doms.npz
78
file 9 : train_t

0.92566132458003481

In [82]:
np.true_divide(np.where(np.argmax(y_real,axis=1)!=predicted_y)[0].size, y_real.shape[0])

0.07433867541996525